Project for the course of Microeconometrics _ Summer 2020, M.Sc. Economics, Bonn University _ [Solmaz Ahmadi](https://github.com/solmazahmadi).

---

 __A replication of Bronzini, R., & Lachini, E. (2014).__

In this notebook, we replicate the key results of the following paper:

>  ***Bronzini, R., & Lachini, E. (2014). Are incentives for R&D effective? Evidence from a regression discontinuity approach. American Economic Journal: Economic Policy, 6(4), 100-134.***


__Downloading and viewing this notebook:__

- The best way to view this notebook is by downloading it and the repository it is located in from [GitHub](https://github.com/solmazahmadi/microeconometrics-course-project-solmazahmadi). Other viewing options like *MyBinder*  or *NBViewer* may have issues with displaying images or coloring of certain parts (missing images can be viewed in the folder [files]() on GitHub).

- The original paper, as well as the data and code provided by the authors can be accessed [here](https://www.aeaweb.org/articles?id=10.1257/pol.6.4.100).

__Information about replication and individual contributions:__

- for the ease of camparision, we preserve the paper's original structure. All sections are titled the same and tables and figures are named and labeled as they appear in Bronzini, R., & Iachini, E. (2014).

- The tables in my replication appear transposed compared to the original tables to suit my workflow in Python.

- For transparency, all sections in the replication that constitute independent contributions by me and are not part of results presented (or include deviations from the methods used) in the paper are marked as extensions.

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#Replication-of-Bronzini,-R.,-&amp;-Iachini,-E.-(2014)." data-toc-modified-id="Replication-of-Bronzini,-R.,-&amp;-Iachini,-E.-(2014).-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span><strong>Replication of Bronzini, R., &amp; Iachini, E. (2014).</strong></a></span></li></ul></li><li><span><a href="#Table-of-Contents" data-toc-modified-id="Table-of-Contents-1"><span class="toc-item-num">1&nbsp;&nbsp;</span><strong>Table of Contents</strong></a></span></li></ul></div>

# 1. Introduction

Bronzini and Lachini (2014) study the effects of a unique R&D subsidy program executed in northern Italy on investment expenditures of firms. The public R&D funding is a government policy that aims to trigger marginal projects, those that would not be carried out without the subventions. The economic rationale behind the R&D subsidies is firstly to conquer market failure of knowledge as a public good the positive externalities of which cannot be fully internalized by the firm. secondly, the government grants aid to overcome the firm's liquidity constraints. Firms were asked to present a proposal for their new projects and an independent technical committee scores them. Only firms whose scores were exceeded a specific threshold were awarded public grants. To estimate the causal impact of subsidies, Bronzini and Lachini (2014) apply a sharp regression discontinuity design (RDD) comparing the private investment spending of funded and nonfunded firms with scores close to the threshold. Among nonexperimental econometric methods, the regression discontinuity controls preferably for the endogeneity of treatment since it can be shown as a randomized experiment by arguing that the agents had been randomly drawn just below or just above the cutoff. The paper finds for the whole sample of firms, the investment expenditures do not increase significantly. since the overall impact hides the considerable heterogeneity in the program’s effect, Bronzini and Lachini (2014)  divide the sample by small and large firms and demonstrate that although the subsidy did not affect large enterprises' investment spending, small companies raised their investments—by roughly the amount of the grant they gained.


Main variables for the sample as a whole are presented in the table 1.1:

| main variables   |            main outcome variables            |      main covriates    |  
|----------------- |----------------------------------------------|----------------------- |
| treatment effect | investment (total, tangible, and intangible) |      score             |   
|        -         |              labor costs                     |       ROA              |   
|        -         |             employmentand                    | Net worth assets/Debts |   
|        -         |                 wages                        |   Cash flow/Sales      |   
|        -         |            service costs                     |  Interest costs/Debts  |   


(((
 By letting the outcome variable be a function of the score, the average treatment
effect of the program is assessed through the estimated value of the discontinuity at
the threshold. ))))

The rest of notbook is structured as follows. In the second section, we review the theoretical framework for the paper. Sectin 3 describes the identification strategy utilized by the authors to unravel causal effects of gorment subsidy program on investment expenditure of companies. Section 4 discusses the paper emprical method used for the estimation. In section 5, we replicate the key results of the paper by Bronzini and Lachini (2014) and discuss it in detail. Section 6 conducts and evaluates the results using multiple robustness checks. finaly, last section colcludes remarks.

# add from 13.06

In [1]:
import numpy as np
import pandas as pd
import scipy
import pandas.io.formats.style
import seaborn as sns
import statsmodels as sm
import statsmodels.formula.api as smf
import statsmodels.formula.api as smf
import statsmodels.api as sm
import matplotlib.pyplot as plt
from IPython.display import HTML
from statsmodels.formula.api import ols
from statsmodels.formula.api import ols
from statsmodels.sandbox.regression.predstd import wls_prediction_std

In [2]:
import econtools.metrics as mt

In [3]:
from auxiliary_get_key_varaibles import get_key_variables

In [4]:
df = get_key_variables()

In [5]:
df

,K,KT,KINT,SALES,ASSETS,INV,INVT,INVINT,LINV,LINVT,...,AGE,WAGE,LEMPL,LWAGE,SIZE,score,treatment,T,s,streat
Firm,,,,,,,,,,,,,,,,,,,,,
0,21237,19339,1898,80721,159591,9707,9318,389,11.640,10.875,...,1974.1232,49.900,7.424,3.910,large,85,treated,1.0,10,10.0
1,1646,1487,159,3707,3381,373,75,298,11.554,10.682,...,NaN,NaN,NaN,NaN,small,82,treated,1.0,7,7.0
2,6931,6728,203,25301,21127,1545,1216,329,11.566,10.708,...,1967.1016,38.683,5.293,3.655,large,89,treated,1.0,14,14.0
3,1131,1115,16,12143,10842,-114,-114,0,11.550,10.678,...,1966.1020,37.397,5.242,3.622,large,70,untreated,0.0,-5,-0.0
4,1691,1649,42,5263,7688,235,219,16,11.553,10.686,...,1969.1120,23.836,4.804,3.171,small,85,treated,1.0,10,10.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
352,1126,985,141,30938,22915,-63,-49,-14,11.550,10.680,...,1995.0107,NaN,NaN,NaN,large,77,treated,1.0,2,2.0
353,494378,185503,308875,773040,1123660,-103863,14641,-118504,0.000,10.971,...,1995.0316,65.707,8.153,4.185,large,88,treated,1.0,13,13.0
354,41568,40187,1381,161396,272656,-10065,-10486,421,11.449,10.405,...,1995.0718,41.551,7.762,3.727,large,84,treated,1.0,9,9.0


In [6]:
ols_pol0 = ols("INVSALES ~ T", data= df).fit()
ols_pol0_coeff = ols_pol0.params.copy()

ols_pol0_coeff[1].round(3)

0.012

In [7]:
ols_pol1 = ols("INVSALES ~ T + s +streat", data= df).fit()
ols_pol1_coeff = ols_pol1.params.copy()

ols_pol1_coeff.round(3)

Intercept    0.021
T            0.040
s           -0.001
streat      -0.001
dtype: float64

In [8]:
#ols_pol1 = ols("INVSALES ~ T + treatment", data= df).fit()
#ols_pol1_coeff = ols_pol1.params.copy()

#ols_pol1_coeff[1].round(3)

In [9]:
ols_pol1 = ols("INVSALES ~ T + s +streat", data= df).fit().summary()
ols_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.563
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.198
Time:                        21:34:27   Log-Likelihood:                 303.35
No. Observations:                 357   AIC:                            -598.7
Df Residuals:                     353   BIC:                            -583.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0205      0.015      1.382      0.168      -0.009       0.050
T              0.0401      0.019      2.102      0.036       0.003       0.078
s             -0.0009      0.001     -1.192      0.234      -0.002       0.001
streat        -0.0007      0.001     -0.541      0.589      -0.003       0.002
==============================================================================
Omnibus:                       63.335   Durbin-Watson:                   1.817
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              101.929
Skew:                           1.057   Prob(JB):                     7.35e-23
Kurtosis:                       4.545   Cond. No.                         64.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [10]:
df["NT"]  = 1-df["T"]

In [11]:
df["s2"]= df["s"].pow(2)
df["s3"]= df["s"].pow(3)
df["streat"] = df["s"]* df["T"]
df["s2treat"] = df["s2"]* df["T"]
df["s3treat"] = df["s3"]* df["T"]
df["snotreat"] = df["s"]* df["NT"]
df["s2notreat"] = df["s2"]* df["NT"]
df["s3notreat"] = df["s3"]* df["NT"]
#ols_pol2 = ols("INVSALES ~ T + s +streat + s2treat", data= df).fit().summary()
#ols_pol2

In [12]:
ols_pol0 =ols("INVSALES ~ C(T) ", data= df).fit().summary()

ols_pol1 = ols("INVSALES ~ C(T) + streat + snotreat ", data= df).fit().summary()

ols_pol2 = ols("INVSALES ~ T + streat + snotreat + s2treat + s2notreat", data= df).fit().summary()

ols_pol3 = ols("INVSALES ~ T + streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data= df).fit().summary()

In [13]:
ols_pol00 = ols("INVTSALES ~ T ", data= df).fit().summary()

ols_pol11 = ols("INVTSALES ~ T + streat+ snotreat ", data= df).fit().summary()

ols_pol22 = ols("INVTSALES ~ T + streat + snotreat + s2treat + s2notreat", data= df).fit().summary()

ols_pol33 =ols("INVTSALES ~ T + streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data= df).fit().summary()

In [14]:
ols_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.003
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.317
Time:                        21:34:27   Log-Likelihood:                 301.50
No. Observations:                 357   AIC:                            -599.0
Df Residuals:                     355   BIC:                            -591.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0333      0.010      3.244      0.001       0.013       0.054
C(T)[T.1.0]     0.0122      0.012      1.002      0.317      -0.012       0.036
==============================================================================
Omnibus:                       63.908   Durbin-Watson:                   1.825
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              103.114
Skew:                           1.064   Prob(JB):                     4.07e-23
Kurtosis:                       4.550   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [15]:
ols_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.563
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.198
Time:                        21:34:27   Log-Likelihood:                 303.35
No. Observations:                 357   AIC:                            -598.7
Df Residuals:                     353   BIC:                            -583.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0205      0.015      1.382      0.168      -0.009       0.050
C(T)[T.1.0]     0.0401      0.019      2.102      0.036       0.003       0.078
streat         -0.0016      0.001     -1.503      0.134      -0.004       0.001
snotreat       -0.0009      0.001     -1.192      0.234      -0.002       0.001
==============================================================================
Omnibus:                       63.335   Durbin-Watson:                   1.817
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              101.929
Skew:                           1.057   Prob(JB):                     7.35e-23
Kurtosis:                       4.545   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [16]:
ols_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.165
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.326
Time:                        21:34:27   Log-Likelihood:                 303.93
No. Observations:                 357   AIC:                            -595.9
Df Residuals:                     351   BIC:                            -572.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0063      0.023      0.269      0.788      -0.040       0.052
T              0.0459      0.029      1.601      0.110      -0.010       0.102
streat         0.0011      0.004      0.273      0.785      -0.007       0.009
snotreat      -0.0035      0.003     -1.049      0.295      -0.010       0.003
s2treat       -0.0001      0.000     -0.724      0.470      -0.001       0.000
s2notreat  -5.681e-05   7.18e-05     -0.791      0.429      -0.000    8.44e-05
==============================================================================
Omnibus:                       62.091   Durbin-Watson:                   1.814
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               98.726
Skew:                           1.045   Prob(JB):                     3.65e-22
Kurtosis:                       4.507   Cond. No.                     2.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [17]:
ols_pol3

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.9476
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.470
Time:                        21:34:27   Log-Likelihood:                 304.35
No. Observations:                 357   AIC:                            -592.7
Df Residuals:                     349   BIC:                            -561.7
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0010      0.036     -0.029      0.977      -0.072       0.069
T              0.0647      0.042      1.552      0.122      -0.017       0.147
streat        -0.0061      0.009     -0.668      0.505      -0.024       0.012
snotreat      -0.0055      0.008     -0.659      0.511      -0.022       0.011
s2treat        0.0007      0.001      0.717      0.474      -0.001       0.003
s2notreat     -0.0002      0.000     -0.388      0.698      -0.001       0.001
s3treat    -2.763e-05   3.19e-05     -0.868      0.386   -9.03e-05     3.5e-05
s3notreat  -1.855e-06   6.89e-06     -0.269      0.788   -1.54e-05    1.17e-05
==============================================================================
Omnibus:                       61.047   Durbin-Watson:                   1.813
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               95.790
Skew:                           1.037   Prob(JB):                     1.58e-21
Kurtosis:                       4.462   Cond. No.                     1.53e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [18]:
ols_pol00 = ols("INVTSALES ~ T ", data= df).fit().summary()

ols_pol11 = ols("INVTSALES ~ T + streat+ snotreat ", data= df).fit().summary()

ols_pol22 = ols("INVTSALES ~ T + streat + snotreat + s2treat + s2notreat", data= df).fit().summary()

ols_pol33 =ols("INVTSALES ~ T + streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data= df).fit().summary()


In [19]:
ols_pol00

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.001
Method:                 Least Squares   F-statistic:                    0.5741
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.449
Time:                        21:34:27   Log-Likelihood:                 357.34
No. Observations:                 357   AIC:                            -710.7
Df Residuals:                     355   BIC:                            -702.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0211      0.009      2.404      0.017       0.004       0.038
T              0.0079      0.010      0.758      0.449      -0.013       0.028
==============================================================================
Omnibus:                       87.252   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              426.189
Skew:                           0.933   Prob(JB):                     2.85e-93
Kurtosis:                       8.017   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [20]:
ols_pol11

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7992
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.495
Time:                        21:34:27   Log-Likelihood:                 358.26
No. Observations:                 357   AIC:                            -708.5
Df Residuals:                     353   BIC:                            -693.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0098      0.013      0.772      0.440      -0.015       0.035
T              0.0241      0.016      1.471      0.142      -0.008       0.056
streat        -0.0005      0.001     -0.566      0.571      -0.002       0.001
snotreat      -0.0008      0.001     -1.226      0.221      -0.002       0.000
==============================================================================
Omnibus:                       86.985   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              417.404
Skew:                           0.936   Prob(JB):                     2.30e-91
Kurtosis:                       7.956   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [21]:
ols_pol22

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.4801
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.791
Time:                        21:34:27   Log-Likelihood:                 358.27
No. Observations:                 357   AIC:                            -704.5
Df Residuals:                     351   BIC:                            -681.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0114      0.020      0.568      0.570      -0.028       0.051
T              0.0217      0.025      0.882      0.378      -0.027       0.070
streat        -0.0003      0.003     -0.083      0.934      -0.007       0.006
snotreat      -0.0005      0.003     -0.190      0.849      -0.006       0.005
s2treat    -1.289e-05      0.000     -0.079      0.937      -0.000       0.000
s2notreat   6.183e-06   6.17e-05      0.100      0.920      -0.000       0.000
==============================================================================
Omnibus:                       86.779   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              417.829
Skew:                           0.932   Prob(JB):                     1.86e-91
Kurtosis:                       7.961   Cond. No.                     2.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [22]:
ols_pol33 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.013
Method:                 Least Squares   F-statistic:                    0.3513
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.929
Time:                        21:34:27   Log-Likelihood:                 358.30
No. Observations:                 357   AIC:                            -700.6
Df Residuals:                     349   BIC:                            -669.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0111      0.031      0.359      0.720      -0.050       0.072
T              0.0251      0.036      0.700      0.484      -0.045       0.096
streat        -0.0022      0.008     -0.278      0.781      -0.018       0.013
snotreat      -0.0006      0.007     -0.087      0.931      -0.015       0.014
s2treat        0.0002      0.001      0.249      0.804      -0.002       0.002
s2notreat   6.942e-07      0.000      0.002      0.999      -0.001       0.001
s3treat    -7.339e-06   2.74e-05     -0.268      0.789   -6.12e-05    4.65e-05
s3notreat  -8.222e-08   5.92e-06     -0.014      0.989   -1.17e-05    1.16e-05
==============================================================================
Omnibus:                       86.523   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              419.134
Skew:                           0.927   Prob(JB):                     9.69e-92
Kurtosis:                       7.974   Cond. No.                     1.53e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [23]:
ols_pol000 = ols("INVINTSALES ~ T ", data= df).fit().summary()

ols_pol111 = ols("INVINTSALES ~ T + streat+ snotreat ", data= df).fit().summary()

ols_pol222 = ols("INVINTSALES ~ T + streat + snotreat + s2treat + s2notreat", data= df).fit().summary()

ols_pol333 =ols("INVINTSALES ~ T + streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data= df).fit().summary()

In [24]:
ols_pol000

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.3569
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.551
Time:                        21:34:27   Log-Likelihood:                 491.83
No. Observations:                 357   AIC:                            -979.7
Df Residuals:                     355   BIC:                            -971.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0122      0.006      2.023      0.044       0.000       0.024
T              0.0043      0.007      0.597      0.551      -0.010       0.018
==============================================================================
Omnibus:                      173.982   Durbin-Watson:                   1.785
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1109.843
Skew:                           1.968   Prob(JB):                    1.00e-241
Kurtosis:                      10.689   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
ols_pol111

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.010
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     1.131
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.336
Time:                        21:34:27   Log-Likelihood:                 493.36
No. Observations:                 357   AIC:                            -978.7
Df Residuals:                     353   BIC:                            -963.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0108      0.009      1.234      0.218      -0.006       0.028
T              0.0159      0.011      1.417      0.157      -0.006       0.038
streat        -0.0011      0.001     -1.728      0.085      -0.002       0.000
snotreat      -0.0001      0.000     -0.227      0.821      -0.001       0.001
==============================================================================
Omnibus:                      172.293   Durbin-Watson:                   1.762
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1086.302
Skew:                           1.949   Prob(JB):                    1.30e-236
Kurtosis:                      10.605   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [26]:
ols_pol222

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.378
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.232
Time:                        21:34:27   Log-Likelihood:                 495.12
No. Observations:                 357   AIC:                            -978.2
Df Residuals:                     351   BIC:                            -955.0
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0050      0.014     -0.367      0.714      -0.032       0.022
T              0.0241      0.017      1.436      0.152      -0.009       0.057
streat         0.0013      0.002      0.581      0.562      -0.003       0.006
snotreat      -0.0029      0.002     -1.510      0.132      -0.007       0.001
s2treat       -0.0001      0.000     -1.112      0.267      -0.000    9.55e-05
s2notreat    -6.3e-05    4.2e-05     -1.499      0.135      -0.000    1.97e-05
==============================================================================
Omnibus:                      171.283   Durbin-Watson:                   1.734
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1085.343
Skew:                           1.932   Prob(JB):                    2.09e-236
Kurtosis:                      10.618   Cond. No.                     2.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [27]:
ols_pol333

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.023
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.182
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.313
Time:                        21:34:27   Log-Likelihood:                 495.83
No. Observations:                 357   AIC:                            -975.7
Df Residuals:                     349   BIC:                            -944.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0121      0.021     -0.575      0.566      -0.053       0.029
T              0.0396      0.024      1.624      0.105      -0.008       0.088
streat        -0.0040      0.005     -0.740      0.460      -0.014       0.007
snotreat      -0.0049      0.005     -0.999      0.318      -0.015       0.005
s2treat        0.0005      0.001      0.866      0.387      -0.001       0.002
s2notreat     -0.0002      0.000     -0.669      0.504      -0.001       0.000
s3treat    -2.037e-05   1.86e-05     -1.093      0.275    -5.7e-05    1.63e-05
s3notreat  -1.785e-06   4.03e-06     -0.443      0.658   -9.71e-06    6.14e-06
==============================================================================
Omnibus:                      170.192   Durbin-Watson:                   1.733
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1063.238
Skew:                           1.923   Prob(JB):                    1.32e-231
Kurtosis:                      10.529   Cond. No.                     1.53e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [28]:
df_wide_wind = df.loc[(52 <= df.score) & (df.score <= 80), :]
df_wide_wind 

,K,KT,KINT,SALES,ASSETS,INV,INVT,INVINT,LINV,LINVT,...,s,streat,NT,s2,s3,s2treat,s3treat,snotreat,s2notreat,s3notreat
Firm,,,,,,,,,,,,,,,,,,,,,
3,1131,1115,16,12143,10842,-114,-114,0,11.550,10.678,...,-5,-0.0,1.0,25,-125,0.0,-0.0,-5.0,25.0,-125.0
7,3788,3788,0,12071,14032,987,677,310,11.560,10.696,...,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
9,2070,1321,749,10718,11990,2770,3156,-386,11.577,10.751,...,1,1.0,0.0,1,1,1.0,1.0,0.0,0.0,0.0
15,17152,17049,103,30226,54628,2275,1805,470,11.573,10.721,...,2,2.0,0.0,4,8,4.0,8.0,0.0,0.0,0.0
17,18017,15318,2699,341343,282072,-203,-1760,1557,11.549,10.639,...,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
344,298,252,46,1034,1189,353,-36,389,11.554,10.680,...,1,1.0,0.0,1,1,1.0,1.0,0.0,0.0,0.0
345,48,28,20,11796,12094,453,158,295,11.555,10.684,...,-7,-0.0,1.0,49,-343,0.0,-0.0,-7.0,49.0,-343.0
350,11161,10510,651,73763,75434,1549,2057,-508,11.566,10.727,...,5,5.0,0.0,25,125,25.0,125.0,0.0,0.0,0.0


In [29]:
ww_pol0 = ols("INVSALES ~ C(T) ", data = df_wide_wind).fit().summary()

ww_pol1 = ols("INVSALES ~ C(T) + streat + snotreat ", data = df_wide_wind).fit().summary()

ww_pol2 = ols("INVSALES ~ T + streat + snotreat + s2treat + s2notreat", data = df_wide_wind).fit().summary()



In [30]:
ww_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     2.688
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.103
Time:                        21:34:28   Log-Likelihood:                 140.52
No. Observations:                 171   AIC:                            -277.0
Df Residuals:                     169   BIC:                            -270.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0280      0.012      2.354      0.020       0.005       0.051
C(T)[T.1.0]     0.0269      0.016      1.639      0.103      -0.005       0.059
==============================================================================
Omnibus:                       34.917   Durbin-Watson:                   1.876
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               54.944
Skew:                           1.070   Prob(JB):                     1.17e-12
Kurtosis:                       4.770   Cond. No.                         2.68
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [31]:
ww_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.126
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.340
Time:                        21:34:28   Log-Likelihood:                 140.89
No. Observations:                 171   AIC:                            -273.8
Df Residuals:                     167   BIC:                            -261.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0127      0.022      0.584      0.560      -0.030       0.056
C(T)[T.1.0]     0.0417      0.031      1.365      0.174      -0.019       0.102
streat          0.0002      0.007      0.025      0.980      -0.013       0.014
snotreat       -0.0020      0.002     -0.843      0.401      -0.007       0.003
==============================================================================
Omnibus:                       33.970   Durbin-Watson:                   1.879
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.151
Skew:                           1.058   Prob(JB):                     4.74e-12
Kurtosis:                       4.686   Cond. No.                         27.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [32]:
ww_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.019
Method:                 Least Squares   F-statistic:                     1.653
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.149
Time:                        21:34:28   Log-Likelihood:                 143.36
No. Observations:                 171   AIC:                            -274.7
Df Residuals:                     165   BIC:                            -255.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0219      0.039     -0.564      0.573      -0.098       0.055
T              0.1102      0.048      2.313      0.022       0.016       0.204
streat        -0.0447      0.024     -1.835      0.068      -0.093       0.003
snotreat      -0.0115      0.009     -1.256      0.211      -0.030       0.007
s2treat        0.0087      0.005      1.918      0.057      -0.000       0.018
s2notreat     -0.0005      0.000     -1.073      0.285      -0.001       0.000
==============================================================================
Omnibus:                       30.908   Durbin-Watson:                   1.880
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.900
Skew:                           1.020   Prob(JB):                     2.93e-10
Kurtosis:                       4.415   Cond. No.                         692.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [33]:
ww_pol00 = ols("INVTSALES ~ T ", data = df_wide_wind).fit().summary()

ww_pol11 = ols("INVTSALES ~ T + streat+ snotreat ", data = df_wide_wind).fit().summary()

ww_pol22 = ols("INVTSALES ~ T + streat + snotreat + s2treat + s2notreat", data = df_wide_wind).fit().summary()



In [34]:
ww_pol00

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     2.175
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.142
Time:                        21:34:28   Log-Likelihood:                 178.92
No. Observations:                 171   AIC:                            -353.8
Df Residuals:                     169   BIC:                            -347.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0157      0.009      1.651      0.101      -0.003       0.034
T              0.0193      0.013      1.475      0.142      -0.007       0.045
==============================================================================
Omnibus:                       74.009   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              274.311
Skew:                           1.680   Prob(JB):                     2.72e-60
Kurtosis:                       8.216   Cond. No.                         2.68
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [35]:
ols_pol11

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7992
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.495
Time:                        21:34:27   Log-Likelihood:                 358.26
No. Observations:                 357   AIC:                            -708.5
Df Residuals:                     353   BIC:                            -693.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0098      0.013      0.772      0.440      -0.015       0.035
T              0.0241      0.016      1.471      0.142      -0.008       0.056
streat        -0.0005      0.001     -0.566      0.571      -0.002       0.001
snotreat      -0.0008      0.001     -1.226      0.221      -0.002       0.000
==============================================================================
Omnibus:                       86.985   Durbin-Watson:                   1.938
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              417.404
Skew:                           0.936   Prob(JB):                     2.30e-91
Kurtosis:                       7.956   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
ols_pol22

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.007
Method:                 Least Squares   F-statistic:                    0.4801
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.791
Time:                        21:34:27   Log-Likelihood:                 358.27
No. Observations:                 357   AIC:                            -704.5
Df Residuals:                     351   BIC:                            -681.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0114      0.020      0.568      0.570      -0.028       0.051
T              0.0217      0.025      0.882      0.378      -0.027       0.070
streat        -0.0003      0.003     -0.083      0.934      -0.007       0.006
snotreat      -0.0005      0.003     -0.190      0.849      -0.006       0.005
s2treat    -1.289e-05      0.000     -0.079      0.937      -0.000       0.000
s2notreat   6.183e-06   6.17e-05      0.100      0.920      -0.000       0.000
==============================================================================
Omnibus:                       86.779   Durbin-Watson:                   1.939
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              417.829
Skew:                           0.932   Prob(JB):                     1.86e-91
Kurtosis:                       7.961   Cond. No.                     2.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [37]:
ww_pol000 = ols("INVINTSALES ~ T ", data = df_wide_wind).fit().summary()

ww_pol111 = ols("INVINTSALES ~ T + streat + snotreat ", data = df_wide_wind).fit().summary()

ww_pol222 = ols("INVINTSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data = df_wide_wind).fit().summary()



In [38]:
ww_pol000

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.6169
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.433
Time:                        21:34:28   Log-Likelihood:                 233.69
No. Observations:                 171   AIC:                            -463.4
Df Residuals:                     169   BIC:                            -457.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0123      0.007      1.790      0.075      -0.001       0.026
T              0.0075      0.010      0.785      0.433      -0.011       0.026
==============================================================================
Omnibus:                       81.703   Durbin-Watson:                   1.664
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              433.646
Skew:                           1.716   Prob(JB):                     6.84e-95
Kurtosis:                      10.006   Cond. No.                         2.68
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [39]:
ww_pol111

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                    0.6942
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.557
Time:                        21:34:28   Log-Likelihood:                 234.44
No. Observations:                 171   AIC:                            -460.9
Df Residuals:                     167   BIC:                            -448.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0012      0.013      0.092      0.926      -0.024       0.026
T              0.0248      0.018      1.401      0.163      -0.010       0.060
streat        -0.0023      0.004     -0.579      0.563      -0.010       0.006
snotreat      -0.0015      0.001     -1.064      0.289      -0.004       0.001
==============================================================================
Omnibus:                       79.630   Durbin-Watson:                   1.645
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              406.793
Skew:                           1.678   Prob(JB):                     4.63e-89
Kurtosis:                       9.770   Cond. No.                         27.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [40]:
ww_pol222

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     1.570
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.171
Time:                        21:34:28   Log-Likelihood:                 237.36
No. Observations:                 171   AIC:                            -462.7
Df Residuals:                     165   BIC:                            -443.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0304      0.022     -1.358      0.176      -0.075       0.014
T              0.0735      0.027      2.674      0.008       0.019       0.128
streat        -0.0250      0.014     -1.782      0.077      -0.053       0.003
snotreat      -0.0101      0.005     -1.917      0.057      -0.021       0.000
s2treat        0.0044      0.003      1.686      0.094      -0.001       0.010
s2notreat     -0.0004      0.000     -1.697      0.092      -0.001    6.77e-05
==============================================================================
Omnibus:                       81.323   Durbin-Watson:                   1.624
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              442.428
Skew:                           1.695   Prob(JB):                     8.47e-97
Kurtosis:                      10.114   Cond. No.                         692.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [41]:
df_narr_wind = df.loc[(66 <= df.score) & (df.score <= 78), :]

In [42]:
df_narr_wind

,K,KT,KINT,SALES,ASSETS,INV,INVT,INVINT,LINV,LINVT,...,s,streat,NT,s2,s3,s2treat,s3treat,snotreat,s2notreat,s3notreat
Firm,,,,,,,,,,,,,,,,,,,,,
3,1131,1115,16,12143,10842,-114,-114,0,11.550,10.678,...,-5,-0.0,1.0,25,-125,0.0,-0.0,-5.0,25.0,-125.0
7,3788,3788,0,12071,14032,987,677,310,11.560,10.696,...,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
9,2070,1321,749,10718,11990,2770,3156,-386,11.577,10.751,...,1,1.0,0.0,1,1,1.0,1.0,0.0,0.0,0.0
15,17152,17049,103,30226,54628,2275,1805,470,11.573,10.721,...,2,2.0,0.0,4,8,4.0,8.0,0.0,0.0,0.0
17,18017,15318,2699,341343,282072,-203,-1760,1557,11.549,10.639,...,0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
339,129,104,25,2398,1475,67,82,-15,11.551,10.683,...,-8,-0.0,1.0,64,-512,0.0,-0.0,-8.0,64.0,-512.0
344,298,252,46,1034,1189,353,-36,389,11.554,10.680,...,1,1.0,0.0,1,1,1.0,1.0,0.0,0.0,0.0
345,48,28,20,11796,12094,453,158,295,11.555,10.684,...,-7,-0.0,1.0,49,-343,0.0,-0.0,-7.0,49.0,-343.0


In [43]:
nw_pol0 = ols("INVSALES ~ C(T) ", data = df_narr_wind).fit().summary()

nw_pol1 = ols("INVSALES ~ C(T) + streat + snotreat ", data = df_narr_wind).fit().summary()

nw_pol2 = ols("INVSALES ~ T + streat + s2treat + snotreat +s2notreat ", data = df_narr_wind).fit().summary()

In [44]:
nw_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.027
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     3.112
Date:                Tue, 23 Jun 2020   Prob (F-statistic):             0.0804
Time:                        21:34:28   Log-Likelihood:                 102.17
No. Observations:                 115   AIC:                            -200.3
Df Residuals:                     113   BIC:                            -194.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0167      0.013      1.266      0.208      -0.009       0.043
C(T)[T.1.0]     0.0330      0.019      1.764      0.080      -0.004       0.070
==============================================================================
Omnibus:                       14.142   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               16.256
Skew:                           0.741   Prob(JB):                     0.000295
Kurtosis:                       4.095   Cond. No.                         2.61
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [45]:
nw_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.022
Method:                 Least Squares   F-statistic:                     1.846
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.143
Time:                        21:34:28   Log-Likelihood:                 103.41
No. Observations:                 115   AIC:                            -198.8
Df Residuals:                     111   BIC:                            -187.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0126      0.035      0.361      0.719      -0.056       0.082
C(T)[T.1.0]     0.0681      0.042      1.611      0.110      -0.016       0.152
streat         -0.0192      0.012     -1.549      0.124      -0.044       0.005
snotreat       -0.0008      0.007     -0.128      0.898      -0.014       0.012
==============================================================================
Omnibus:                       12.394   Durbin-Watson:                   1.960
Prob(Omnibus):                  0.002   Jarque-Bera (JB):               13.475
Skew:                           0.697   Prob(JB):                      0.00119
Kurtosis:                       3.933   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [46]:
nw_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.088
Model:                            OLS   Adj. R-squared:                  0.046
Method:                 Least Squares   F-statistic:                     2.099
Date:                Tue, 23 Jun 2020   Prob (F-statistic):             0.0709
Time:                        21:34:28   Log-Likelihood:                 105.90
No. Observations:                 115   AIC:                            -199.8
Df Residuals:                     109   BIC:                            -183.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1559      0.074      2.100      0.038       0.009       0.303
T             -0.0791      0.079     -0.999      0.320      -0.236       0.078
streat        -0.0085      0.041     -0.205      0.838      -0.090       0.073
s2treat       -0.0036      0.013     -0.271      0.787      -0.030       0.023
snotreat       0.0645      0.031      2.102      0.038       0.004       0.125
s2notreat      0.0063      0.003      2.178      0.032       0.001       0.012
==============================================================================
Omnibus:                       11.922   Durbin-Watson:                   1.966
Prob(Omnibus):                  0.003   Jarque-Bera (JB):               12.651
Skew:                           0.695   Prob(JB):                      0.00179
Kurtosis:                       3.840   Cond. No.                         308.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [47]:
nw_pol00 = ols("INVTSALES ~ C(T) ", data = df_narr_wind).fit().summary()

nw_pol11 = ols("INVTSALES ~ C(T) + streat + snotreat ", data = df_narr_wind).fit().summary()

nw_pol22 = ols("INVTSALES ~ T + streat + snotreat + s2treat + s2notreat", data = df_narr_wind).fit().summary()

In [48]:
nw_pol00

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     2.540
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.114
Time:                        21:34:28   Log-Likelihood:                 135.44
No. Observations:                 115   AIC:                            -266.9
Df Residuals:                     113   BIC:                            -261.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0106      0.010      1.074      0.285      -0.009       0.030
C(T)[T.1.0]     0.0223      0.014      1.594      0.114      -0.005       0.050
==============================================================================
Omnibus:                       24.839   Durbin-Watson:                   1.956
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               43.550
Skew:                           0.942   Prob(JB):                     3.49e-10
Kurtosis:                       5.354   Cond. No.                         2.61
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [49]:
nw_pol11

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.070
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.365
Time:                        21:34:29   Log-Likelihood:                 135.80
No. Observations:                 115   AIC:                            -263.6
Df Residuals:                     111   BIC:                            -252.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0289      0.026      1.102      0.273      -0.023       0.081
C(T)[T.1.0]     0.0095      0.032      0.297      0.767      -0.054       0.073
streat         -0.0034      0.009     -0.361      0.719      -0.022       0.015
snotreat        0.0037      0.005      0.754      0.453      -0.006       0.013
==============================================================================
Omnibus:                       25.510   Durbin-Watson:                   2.005
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               47.309
Skew:                           0.939   Prob(JB):                     5.33e-11
Kurtosis:                       5.519   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [50]:
nw_pol22 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              INVTSALES   R-squared:                       0.054
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.239
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.296
Time:                        21:34:29   Log-Likelihood:                 137.34
No. Observations:                 115   AIC:                            -262.7
Df Residuals:                     109   BIC:                            -246.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1134      0.056      2.008      0.047       0.001       0.225
T             -0.0787      0.060     -1.305      0.195      -0.198       0.041
streat         0.0067      0.031      0.212      0.833      -0.056       0.069
snotreat       0.0422      0.023      1.809      0.073      -0.004       0.089
s2treat       -0.0033      0.010     -0.334      0.739      -0.023       0.016
s2notreat      0.0037      0.002      1.688      0.094      -0.001       0.008
==============================================================================
Omnibus:                       24.095   Durbin-Watson:                   2.030
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               40.794
Skew:                           0.931   Prob(JB):                     1.39e-09
Kurtosis:                       5.247   Cond. No.                         308.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [51]:
nw_pol000 = ols("INVINTSALES ~ C(T) ", data = df_narr_wind).fit().summary()

nw_pol111 = ols("INVINTSALES ~  C(T) + streat + snotreat ", data = df_narr_wind).fit().summary()

nw_pol222 = ols("INVINTSALES ~ T + streat + snotreat + s2treat + s2notreat", data = df_narr_wind).fit().summary()

#nw_pol222 = ols("INVINTSALES ~ T + streat + snotreat + s2treat + s2notreat", data = df_narr_wind).fit(cov_type='cluster').summary()

In [52]:
nw_pol000

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.8056
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.371
Time:                        21:34:29   Log-Likelihood:                 154.81
No. Observations:                 115   AIC:                            -305.6
Df Residuals:                     113   BIC:                            -300.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       0.0061      0.008      0.734      0.465      -0.010       0.023
C(T)[T.1.0]     0.0106      0.012      0.898      0.371      -0.013       0.034
==============================================================================
Omnibus:                       67.299   Durbin-Watson:                   1.797
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              416.888
Skew:                           1.865   Prob(JB):                     2.98e-91
Kurtosis:                      11.549   Cond. No.                         2.61
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [53]:
nw_pol111

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.053
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     2.089
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.106
Time:                        21:34:29   Log-Likelihood:                 157.56
No. Observations:                 115   AIC:                            -307.1
Df Residuals:                     111   BIC:                            -296.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.0163      0.022     -0.750      0.455      -0.059       0.027
C(T)[T.1.0]     0.0586      0.026      2.221      0.028       0.006       0.111
streat         -0.0158      0.008     -2.048      0.043      -0.031      -0.001
snotreat       -0.0045      0.004     -1.115      0.267      -0.013       0.004
==============================================================================
Omnibus:                       65.518   Durbin-Watson:                   1.766
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              400.467
Skew:                           1.806   Prob(JB):                     1.10e-87
Kurtosis:                      11.398   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [54]:
nw_pol222

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.071
Model:                            OLS   Adj. R-squared:                  0.028
Method:                 Least Squares   F-statistic:                     1.656
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.151
Time:                        21:34:29   Log-Likelihood:                 158.61
No. Observations:                 115   AIC:                            -305.2
Df Residuals:                     109   BIC:                            -288.8
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0427      0.047      0.910      0.365      -0.050       0.136
T             -0.0006      0.050     -0.012      0.991      -0.100       0.099
streat        -0.0153      0.026     -0.585      0.560      -0.067       0.037
snotreat       0.0224      0.019      1.153      0.251      -0.016       0.061
s2treat       -0.0002      0.008     -0.021      0.983      -0.017       0.016
s2notreat      0.0026      0.002      1.418      0.159      -0.001       0.006
==============================================================================
Omnibus:                       66.755   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              427.866
Skew:                           1.829   Prob(JB):                     1.23e-93
Kurtosis:                      11.713   Cond. No.                         308.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [55]:
IK_pol0 = ols("INVK ~ T ", data = df).fit().summary()

IK_pol1 = ols("INVK ~ T + streat + snotreat ", data = df).fit().summary()

IK_pol2 = ols("INVK ~ T +  +streat + snotreat + s2treat + s2notreat", data = df).fit().summary()

IK_pol3 = ols("INVK ~ T +  +streat + snotreat + s2treat + s2notreat + s3treat +s3notreat", data = df).fit().summary()


In [56]:
IK_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.623
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.204
Time:                        21:34:29   Log-Likelihood:                -597.78
No. Observations:                 357   AIC:                             1200.
Df Residuals:                     355   BIC:                             1207.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3540      0.128      2.775      0.006       0.103       0.605
T              0.1927      0.151      1.274      0.204      -0.105       0.490
==============================================================================
Omnibus:                      351.340   Durbin-Watson:                   2.056
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9611.151
Skew:                           4.296   Prob(JB):                         0.00
Kurtosis:                      26.923   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [57]:
IK_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     2.539
Date:                Tue, 23 Jun 2020   Prob (F-statistic):             0.0564
Time:                        21:34:29   Log-Likelihood:                -594.78
No. Observations:                 357   AIC:                             1198.
Df Residuals:                     353   BIC:                             1213.
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3790      0.184      2.062      0.040       0.018       0.740
T              0.4700      0.236      1.992      0.047       0.006       0.934
streat        -0.0331      0.014     -2.437      0.015      -0.060      -0.006
snotreat       0.0018      0.010      0.188      0.851      -0.017       0.021
==============================================================================
Omnibus:                      345.841   Durbin-Watson:                   2.052
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9045.158
Skew:                           4.208   Prob(JB):                         0.00
Kurtosis:                      26.179   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [58]:
IK_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.024
Model:                            OLS   Adj. R-squared:                  0.010
Method:                 Least Squares   F-statistic:                     1.705
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.133
Time:                        21:34:29   Log-Likelihood:                -594.31
No. Observations:                 357   AIC:                             1201.
Df Residuals:                     351   BIC:                             1224.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.1672      0.289      0.579      0.563      -0.401       0.735
T              0.6584      0.355      1.854      0.065      -0.040       1.357
streat        -0.0255      0.048     -0.527      0.599      -0.121       0.070
snotreat      -0.0359      0.041     -0.880      0.379      -0.116       0.044
s2treat       -0.0004      0.002     -0.164      0.870      -0.005       0.004
s2notreat     -0.0008      0.001     -0.951      0.342      -0.003       0.001
==============================================================================
Omnibus:                      346.778   Durbin-Watson:                   2.045
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9166.885
Skew:                           4.221   Prob(JB):                         0.00
Kurtosis:                      26.345   Cond. No.                     2.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [59]:
IK_pol3

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.009
Method:                 Least Squares   F-statistic:                     1.447
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.185
Time:                        21:34:29   Log-Likelihood:                -593.49
No. Observations:                 357   AIC:                             1203.
Df Residuals:                     349   BIC:                             1234.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0932      0.443     -0.210      0.834      -0.965       0.778
T              1.0831      0.515      2.101      0.036       0.069       2.097
streat        -0.1284      0.113     -1.135      0.257      -0.351       0.094
snotreat      -0.1100      0.104     -1.058      0.291      -0.315       0.095
s2treat        0.0121      0.013      0.958      0.339      -0.013       0.037
s2notreat     -0.0053      0.006     -0.913      0.362      -0.017       0.006
s3treat       -0.0004      0.000     -1.006      0.315      -0.001       0.000
s3notreat  -6.602e-05   8.52e-05     -0.775      0.439      -0.000       0.000
==============================================================================
Omnibus:                      343.742   Durbin-Watson:                   2.053
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             8820.883
Skew:                           4.176   Prob(JB):                         0.00
Kurtosis:                      25.875   Cond. No.                     1.53e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [60]:
IA_pol0 = ols("INVA ~ T ", data = df).fit().summary()

IA_pol1 = ols("INVA  ~ T + streat + snotreat ", data = df).fit().summary()

IA_pol2 = ols("INVA  ~ T +  +streat + snotreat + s2treat + s2notreat", data = df).fit().summary()

IA_pol3 = ols("INVA  ~ T +  +streat + snotreat + s2treat + s2notreat + s3treat +s3notreat", data = df).fit().summary()

In [61]:
IA_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVA   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.025
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.156
Time:                        21:34:29   Log-Likelihood:                 261.45
No. Observations:                 357   AIC:                            -518.9
Df Residuals:                     355   BIC:                            -511.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0331      0.011      2.877      0.004       0.010       0.056
T              0.0194      0.014      1.423      0.156      -0.007       0.046
==============================================================================
Omnibus:                      100.857   Durbin-Watson:                   1.854
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              253.045
Skew:                           1.365   Prob(JB):                     1.13e-55
Kurtosis:                       6.092   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [62]:
IA_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVA   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.484
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.219
Time:                        21:34:29   Log-Likelihood:                 262.68
No. Observations:                 357   AIC:                            -517.4
Df Residuals:                     353   BIC:                            -501.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0232      0.017      1.395      0.164      -0.010       0.056
T              0.0441      0.021      2.065      0.040       0.002       0.086
streat        -0.0016      0.001     -1.324      0.186      -0.004       0.001
snotreat      -0.0007      0.001     -0.819      0.413      -0.002       0.001
==============================================================================
Omnibus:                      102.214   Durbin-Watson:                   1.843
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.307
Skew:                           1.373   Prob(JB):                     1.10e-57
Kurtosis:                       6.178   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [63]:
IA_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVA   R-squared:                       0.021
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.538
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.177
Time:                        21:34:29   Log-Likelihood:                 264.31
No. Observations:                 357   AIC:                            -516.6
Df Residuals:                     351   BIC:                            -493.3
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0072      0.026     -0.276      0.782      -0.058       0.044
T              0.0620      0.032      1.935      0.054      -0.001       0.125
streat         0.0024      0.004      0.554      0.580      -0.006       0.011
snotreat      -0.0061      0.004     -1.664      0.097      -0.013       0.001
s2treat       -0.0002      0.000     -0.966      0.335      -0.001       0.000
s2notreat     -0.0001   8.02e-05     -1.513      0.131      -0.000    3.64e-05
==============================================================================
Omnibus:                       97.410   Durbin-Watson:                   1.834
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              240.737
Skew:                           1.323   Prob(JB):                     5.30e-53
Kurtosis:                       6.030   Cond. No.                     2.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [64]:
IA_pol3

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVA   R-squared:                       0.026
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     1.327
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.237
Time:                        21:34:29   Log-Likelihood:                 265.13
No. Observations:                 357   AIC:                            -514.3
Df Residuals:                     349   BIC:                            -483.2
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0317      0.040     -0.793      0.428      -0.110       0.047
T              0.1009      0.047      2.169      0.031       0.009       0.192
streat        -0.0066      0.010     -0.645      0.519      -0.027       0.013
snotreat      -0.0131      0.009     -1.396      0.164      -0.032       0.005
s2treat        0.0009      0.001      0.778      0.437      -0.001       0.003
s2notreat     -0.0005      0.001     -1.032      0.303      -0.002       0.000
s3treat    -3.471e-05   3.55e-05     -0.976      0.330      -0.000    3.52e-05
s3notreat  -6.215e-06   7.69e-06     -0.808      0.419   -2.13e-05    8.91e-06
==============================================================================
Omnibus:                       95.670   Durbin-Watson:                   1.836
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              231.086
Skew:                           1.310   Prob(JB):                     6.61e-51
Kurtosis:                       5.945   Cond. No.                     1.53e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [65]:
LC_pol0 = ols("LCSALES ~ T ", data = df).fit().summary()

LC_pol1 = ols("LCSALES  ~ T + streat + snotreat ", data = df).fit().summary()

LC_pol2 = ols("LCSALES  ~ T +  +streat + snotreat + s2treat + s2notreat", data = df).fit().summary()

LC_pol3 = ols("LCSALES  ~ T +  +streat + snotreat + s2treat + s2notreat + s3treat +s3notreat", data = df).fit().summary()

In [66]:
LC_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                LCSALES   R-squared:                       0.005
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.644
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.201
Time:                        21:34:29   Log-Likelihood:                -120.10
No. Observations:                 357   AIC:                             244.2
Df Residuals:                     355   BIC:                             252.0
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7346      0.033     21.947      0.000       0.669       0.800
T             -0.0509      0.040     -1.282      0.201      -0.129       0.027
==============================================================================
Omnibus:                       87.034   Durbin-Watson:                   2.022
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              212.560
Skew:                           1.185   Prob(JB):                     6.97e-47
Kurtosis:                       5.945   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [67]:
LC_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                LCSALES   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                 -0.002
Method:                 Least Squares   F-statistic:                    0.7904
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.500
Time:                        21:34:29   Log-Likelihood:                -119.73
No. Observations:                 357   AIC:                             247.5
Df Residuals:                     353   BIC:                             263.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7572      0.049     15.589      0.000       0.662       0.853
T             -0.0549      0.062     -0.881      0.379      -0.178       0.068
streat        -0.0020      0.004     -0.565      0.572      -0.009       0.005
snotreat       0.0016      0.003      0.643      0.520      -0.003       0.007
==============================================================================
Omnibus:                       88.805   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              222.457
Skew:                           1.198   Prob(JB):                     4.94e-49
Kurtosis:                       6.036   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [68]:
LC_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                LCSALES   R-squared:                       0.014
Model:                            OLS   Adj. R-squared:                 -0.000
Method:                 Least Squares   F-statistic:                    0.9662
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.438
Time:                        21:34:29   Log-Likelihood:                -118.48
No. Observations:                 357   AIC:                             249.0
Df Residuals:                     351   BIC:                             272.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8486      0.076     11.138      0.000       0.699       0.998
T             -0.1535      0.094     -1.639      0.102      -0.338       0.031
streat         0.0003      0.013      0.022      0.983      -0.025       0.025
snotreat       0.0179      0.011      1.664      0.097      -0.003       0.039
s2treat       -0.0001      0.001     -0.188      0.851      -0.001       0.001
s2notreat      0.0004      0.000      1.556      0.121   -9.63e-05       0.001
==============================================================================
Omnibus:                       89.659   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              226.454
Skew:                           1.206   Prob(JB):                     6.70e-50
Kurtosis:                       6.067   Cond. No.                     2.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [69]:
LC_pol3

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                LCSALES   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.603
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.133
Time:                        21:34:29   Log-Likelihood:                -115.27
No. Observations:                 357   AIC:                             246.5
Df Residuals:                     349   BIC:                             277.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0569      0.116      9.104      0.000       0.829       1.285
T             -0.3983      0.135     -2.950      0.003      -0.664      -0.133
streat         0.0231      0.030      0.780      0.436      -0.035       0.081
snotreat       0.0772      0.027      2.835      0.005       0.024       0.131
s2treat       -0.0029      0.003     -0.874      0.383      -0.009       0.004
s2notreat      0.0039      0.002      2.581      0.010       0.001       0.007
s3treat     8.802e-05      0.000      0.853      0.394      -0.000       0.000
s3notreat   5.284e-05   2.23e-05      2.368      0.018    8.95e-06    9.67e-05
==============================================================================
Omnibus:                       82.310   Durbin-Watson:                   2.018
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              184.538
Skew:                           1.158   Prob(JB):                     8.47e-41
Kurtosis:                       5.653   Cond. No.                     1.53e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [70]:
SC_pol0 = ols("SCSALES ~ T ", data = df).fit().summary()

SC_pol1 = ols("SCSALES  ~ T + streat + snotreat ", data = df).fit().summary()

SC_pol2 = ols("SCSALES  ~ T +  +streat + snotreat + s2treat + s2notreat", data = df).fit().summary()

SC_pol3 = ols("SCSALES  ~ T +  +streat + snotreat + s2treat + s2notreat + s3treat +s3notreat", data = df).fit().summary()

In [71]:
SC_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                SCSALES   R-squared:                       0.006
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     2.243
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.135
Time:                        21:34:30   Log-Likelihood:                -271.08
No. Observations:                 357   AIC:                             546.2
Df Residuals:                     355   BIC:                             553.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0532      0.051     20.615      0.000       0.953       1.154
T             -0.0907      0.061     -1.498      0.135      -0.210       0.028
==============================================================================
Omnibus:                      177.828   Durbin-Watson:                   1.742
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1112.630
Skew:                           2.032   Prob(JB):                    2.49e-242
Kurtosis:                      10.635   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [72]:
SC_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                SCSALES   R-squared:                       0.015
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     1.780
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.151
Time:                        21:34:30   Log-Likelihood:                -269.52
No. Observations:                 357   AIC:                             547.0
Df Residuals:                     353   BIC:                             562.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9697      0.074     13.123      0.000       0.824       1.115
T              0.0330      0.095      0.347      0.729      -0.154       0.220
streat        -0.0044      0.005     -0.805      0.422      -0.015       0.006
snotreat      -0.0060      0.004     -1.562      0.119      -0.014       0.002
==============================================================================
Omnibus:                      177.686   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1161.972
Skew:                           2.010   Prob(JB):                    4.80e-253
Kurtosis:                      10.871   Cond. No.                         42.9
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [73]:
SC_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                SCSALES   R-squared:                       0.016
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.139
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.339
Time:                        21:34:30   Log-Likelihood:                -269.33
No. Observations:                 357   AIC:                             550.7
Df Residuals:                     351   BIC:                             573.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0220      0.116      8.791      0.000       0.793       1.251
T             -0.0080      0.143     -0.056      0.955      -0.289       0.273
streat        -0.0081      0.019     -0.414      0.679      -0.046       0.030
snotreat       0.0033      0.016      0.200      0.841      -0.029       0.036
s2treat        0.0002      0.001      0.196      0.845      -0.002       0.002
s2notreat      0.0002      0.000      0.584      0.560      -0.000       0.001
==============================================================================
Omnibus:                      177.746   Durbin-Watson:                   1.749
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1166.751
Skew:                           2.009   Prob(JB):                    4.40e-254
Kurtosis:                      10.892   Cond. No.                     2.45e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.45e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [74]:
SC_pol3

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                SCSALES   R-squared:                       0.017
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.8606
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.538
Time:                        21:34:30   Log-Likelihood:                -269.15
No. Observations:                 357   AIC:                             554.3
Df Residuals:                     349   BIC:                             585.3
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0576      0.179      5.920      0.000       0.706       1.409
T             -0.0789      0.208     -0.380      0.704      -0.488       0.330
streat         0.0140      0.046      0.308      0.758      -0.076       0.104
snotreat       0.0134      0.042      0.320      0.749      -0.069       0.096
s2treat       -0.0025      0.005     -0.490      0.624      -0.012       0.008
s2notreat      0.0008      0.002      0.350      0.727      -0.004       0.005
s3treat     8.514e-05      0.000      0.536      0.592      -0.000       0.000
s3notreat   9.036e-06   3.43e-05      0.263      0.793   -5.85e-05    7.66e-05
==============================================================================
Omnibus:                      179.227   Durbin-Watson:                   1.747
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1199.353
Skew:                           2.022   Prob(JB):                    3.66e-261
Kurtosis:                      11.017   Cond. No.                     1.53e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.53e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [75]:
IK_pol0 = ols("INVK ~ T ", data = df_wide_wind).fit().summary()

IK_pol1 = ols("INVK  ~ T + streat + snotreat ", data =  df_wide_wind).fit().summary()

IK_pol2 = ols("INVK  ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_wide_wind).fit().summary()



In [76]:
IK_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.019
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     3.210
Date:                Tue, 23 Jun 2020   Prob (F-statistic):             0.0750
Time:                        21:34:30   Log-Likelihood:                -318.19
No. Observations:                 171   AIC:                             640.4
Df Residuals:                     169   BIC:                             646.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3792      0.174      2.181      0.031       0.036       0.722
T              0.4293      0.240      1.792      0.075      -0.044       0.902
==============================================================================
Omnibus:                      174.192   Durbin-Watson:                   1.927
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2985.047
Skew:                           4.003   Prob(JB):                         0.00
Kurtosis:                      21.838   Cond. No.                         2.68
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [77]:
IK_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.153
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.329
Time:                        21:34:30   Log-Likelihood:                -318.05
No. Observations:                 171   AIC:                             644.1
Df Residuals:                     167   BIC:                             656.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2382      0.318      0.749      0.455      -0.390       0.866
T              0.5628      0.447      1.258      0.210      -0.321       1.446
streat         0.0028      0.100      0.028      0.978      -0.195       0.201
snotreat      -0.0185      0.035     -0.530      0.597      -0.087       0.050
==============================================================================
Omnibus:                      174.642   Durbin-Watson:                   1.930
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3019.506
Skew:                           4.014   Prob(JB):                         0.00
Kurtosis:                      21.956   Cond. No.                         27.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [78]:
IK_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.425
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.218
Time:                        21:34:30   Log-Likelihood:                -316.19
No. Observations:                 171   AIC:                             644.4
Df Residuals:                     165   BIC:                             663.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.3108      0.569     -0.546      0.586      -1.435       0.813
T              1.5043      0.700      2.150      0.033       0.123       2.886
streat        -0.5164      0.358     -1.443      0.151      -1.223       0.190
snotreat      -0.1691      0.134     -1.259      0.210      -0.434       0.096
s2treat        0.1008      0.067      1.511      0.133      -0.031       0.232
s2notreat     -0.0072      0.006     -1.161      0.247      -0.019       0.005
==============================================================================
Omnibus:                      172.746   Durbin-Watson:                   1.968
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2874.651
Skew:                           3.966   Prob(JB):                         0.00
Kurtosis:                      21.454   Cond. No.                         692.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [79]:
IA_pol0 = ols("INVA ~ T ", data = df_wide_wind).fit().summary()

IA_pol1 = ols("INVA  ~ T + streat + snotreat ", data =  df_wide_wind).fit().summary()

IA_pol2 = ols("INVA  ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_wide_wind).fit().summary()


In [80]:
LC_pol0 = ols("LCSALES ~ T ", data = df_wide_wind).fit().summary()

LC_pol1 = ols("LCSALES  ~ T + streat + snotreat ", data =  df_wide_wind).fit().summary()

LC_pol2 = ols("LCSALES  ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_wide_wind).fit().summary()

In [81]:
LC_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                LCSALES   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                  0.009312
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.923
Time:                        21:34:30   Log-Likelihood:                -63.978
No. Observations:                 171   AIC:                             132.0
Df Residuals:                     169   BIC:                             138.2
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.7196      0.039     18.304      0.000       0.642       0.797
T             -0.0052      0.054     -0.097      0.923      -0.112       0.102
==============================================================================
Omnibus:                       46.911   Durbin-Watson:                   2.234
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               95.416
Skew:                           1.259   Prob(JB):                     1.91e-21
Kurtosis:                       5.655   Cond. No.                         2.68
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [82]:
LC_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                LCSALES   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.063
Method:                 Least Squares   F-statistic:                     4.803
Date:                Tue, 23 Jun 2020   Prob (F-statistic):            0.00310
Time:                        21:34:30   Log-Likelihood:                -56.907
No. Observations:                 171   AIC:                             121.8
Df Residuals:                     167   BIC:                             134.4
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9128      0.069     13.210      0.000       0.776       1.049
T             -0.3024      0.097     -3.112      0.002      -0.494      -0.111
streat         0.0390      0.022      1.790      0.075      -0.004       0.082
snotreat       0.0253      0.008      3.346      0.001       0.010       0.040
==============================================================================
Omnibus:                       48.488   Durbin-Watson:                   2.231
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              102.069
Skew:                           1.284   Prob(JB):                     6.85e-23
Kurtosis:                       5.781   Cond. No.                         27.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [83]:
LC_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                LCSALES   R-squared:                       0.095
Model:                            OLS   Adj. R-squared:                  0.068
Method:                 Least Squares   F-statistic:                     3.479
Date:                Tue, 23 Jun 2020   Prob (F-statistic):            0.00513
Time:                        21:34:30   Log-Likelihood:                -55.413
No. Observations:                 171   AIC:                             122.8
Df Residuals:                     165   BIC:                             141.7
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8467      0.124      6.835      0.000       0.602       1.091
T             -0.1470      0.152     -0.965      0.336      -0.448       0.154
streat        -0.0792      0.078     -1.017      0.311      -0.233       0.075
snotreat       0.0071      0.029      0.244      0.807      -0.051       0.065
s2treat        0.0229      0.015      1.580      0.116      -0.006       0.052
s2notreat     -0.0009      0.001     -0.642      0.522      -0.004       0.002
==============================================================================
Omnibus:                       46.049   Durbin-Watson:                   2.266
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               93.182
Skew:                           1.238   Prob(JB):                     5.83e-21
Kurtosis:                       5.636   Cond. No.                         692.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [84]:
SC_pol0 = ols("SCSALES ~ T ", data = df_wide_wind).fit().summary()

SC_pol1 = ols("SCSALES  ~ T + streat + snotreat ", data =  df_wide_wind).fit().summary()

SC_pol2 = ols("SCSALES  ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_wide_wind).fit().summary()

In [85]:
SC_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                SCSALES   R-squared:                       0.000
Model:                            OLS   Adj. R-squared:                 -0.006
Method:                 Least Squares   F-statistic:                  0.008981
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.925
Time:                        21:34:30   Log-Likelihood:                -131.42
No. Observations:                 171   AIC:                             266.8
Df Residuals:                     169   BIC:                             273.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.9997      0.058     17.141      0.000       0.885       1.115
T             -0.0076      0.080     -0.095      0.925      -0.166       0.151
==============================================================================
Omnibus:                      133.535   Durbin-Watson:                   1.987
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1566.584
Skew:                           2.816   Prob(JB):                         0.00
Kurtosis:                      16.717   Cond. No.                         2.68
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [86]:
SC_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                SCSALES   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.2100
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.889
Time:                        21:34:30   Log-Likelihood:                -131.10
No. Observations:                 171   AIC:                             270.2
Df Residuals:                     167   BIC:                             282.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0700      0.107     10.033      0.000       0.859       1.281
T             -0.0770      0.150     -0.513      0.608      -0.373       0.219
streat        -0.0003      0.034     -0.010      0.992      -0.067       0.066
snotreat       0.0092      0.012      0.788      0.432      -0.014       0.032
==============================================================================
Omnibus:                      133.915   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1577.546
Skew:                           2.826   Prob(JB):                         0.00
Kurtosis:                      16.765   Cond. No.                         27.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [87]:
SC_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                SCSALES   R-squared:                       0.018
Model:                            OLS   Adj. R-squared:                 -0.012
Method:                 Least Squares   F-statistic:                    0.6011
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.699
Time:                        21:34:30   Log-Likelihood:                -129.88
No. Observations:                 171   AIC:                             271.8
Df Residuals:                     165   BIC:                             290.6
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.8824      0.191      4.608      0.000       0.504       1.260
T              0.1973      0.235      0.838      0.403      -0.267       0.662
streat        -0.1150      0.120     -0.955      0.341      -0.353       0.123
snotreat      -0.0423      0.045     -0.936      0.351      -0.131       0.047
s2treat        0.0223      0.022      0.992      0.323      -0.022       0.067
s2notreat     -0.0025      0.002     -1.179      0.240      -0.007       0.002
==============================================================================
Omnibus:                      132.591   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1519.453
Skew:                           2.798   Prob(JB):                         0.00
Kurtosis:                      16.488   Cond. No.                         692.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [88]:
IK_pol0 = ols("INVK ~ T ", data = df_narr_wind).fit().summary()

IK_pol1 = ols("INVK  ~ T + streat + snotreat ", data =  df_narr_wind).fit().summary()

IK_pol2 = ols("INVK  ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_narr_wind).fit().summary()

In [89]:
IK_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.362
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.246
Time:                        21:34:31   Log-Likelihood:                -212.03
No. Observations:                 115   AIC:                             428.1
Df Residuals:                     113   BIC:                             433.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.3309      0.203      1.634      0.105      -0.070       0.732
T              0.3358      0.288      1.167      0.246      -0.234       0.906
==============================================================================
Omnibus:                      143.552   Durbin-Watson:                   2.028
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             3197.795
Skew:                           4.595   Prob(JB):                         0.00
Kurtosis:                      27.143   Cond. No.                         2.61
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [90]:
IK_pol1

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.040
Model:                            OLS   Adj. R-squared:                  0.014
Method:                 Least Squares   F-statistic:                     1.550
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.206
Time:                        21:34:31   Log-Likelihood:                -210.36
No. Observations:                 115   AIC:                             428.7
Df Residuals:                     111   BIC:                             439.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.1590      0.533     -0.298      0.766      -1.215       0.897
T              1.2882      0.647      1.991      0.049       0.006       2.571
streat        -0.2866      0.190     -1.512      0.133      -0.662       0.089
snotreat      -0.0990      0.100     -0.993      0.323      -0.297       0.099
==============================================================================
Omnibus:                      138.304   Durbin-Watson:                   2.043
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2744.380
Skew:                           4.384   Prob(JB):                         0.00
Kurtosis:                      25.268   Cond. No.                         21.8
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [91]:
IK_pol2

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   INVK   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     1.269
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.282
Time:                        21:34:31   Log-Likelihood:                -209.46
No. Observations:                 115   AIC:                             430.9
Df Residuals:                     109   BIC:                             447.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0861      1.153      0.075      0.941      -2.199       2.371
T              1.3292      1.230      1.080      0.282      -1.109       3.767
streat        -1.0745      0.642     -1.675      0.097      -2.346       0.197
snotreat       0.0128      0.477      0.027      0.979      -0.932       0.957
s2treat        0.2626      0.204      1.285      0.201      -0.142       0.667
s2notreat      0.0108      0.045      0.240      0.811      -0.078       0.100
==============================================================================
Omnibus:                      137.018   Durbin-Watson:                   2.050
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             2648.312
Skew:                           4.331   Prob(JB):                         0.00
Kurtosis:                      24.855   Cond. No.                         308.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [92]:
IA_pol0 = ols("INVA ~ T ", data = df_narr_wind).fit().summary()

IA_pol1 = ols("INVA   ~ T + streat + snotreat ", data =  df_narr_wind).fit().summary()

IA_pol2 = ols("INVA   ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_narr_wind).fit().summary()

In [93]:
LC_pol0 = ols("LCSALES ~ T ", data = df_narr_wind).fit().summary()

LC_pol1 = ols("LCSALES   ~ T + streat + snotreat ", data =  df_narr_wind).fit().summary()

LC_pol2 = ols("LCSALES   ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_narr_wind).fit().summary()

In [94]:
SC_pol0 = ols("SCSALES ~ T ", data = df_narr_wind).fit().summary()

SC_pol1 = ols("SCSALES   ~ T + streat + snotreat ", data =  df_narr_wind).fit().summary()

SC_pol2 = ols("SCSALES   ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_narr_wind)

In [95]:
#from lmfit import Model

##                                           I. Conceptual Framework and Empirical Evidence

Although theoritically government R&D  subsidies leads to a decline in the capital cost, an increase in investment profitability and subsequently expansion of firms’ R&D investment, the type of project which is funded determines the grant's effectiveness. The policy would be effective if the grant triggers the marginal projects, which would not have been profitable without public funding. However, the program would be ineffective if the grant finances the inframarginal projects, which would have been undertaken even in the absence of public grant because of their profitability. Under these conditions, the substitution of private for public funding to exploit the lower capital cost of public subsidies will happen without any positive impact on R&D investment. 


To assess the program’s effectiveness, we take three considerations. First, Ceteris paribus, the sucess of program might be affected by subsidizing the projects which were regonized as privately financed unprofitable by their companies because the firms face asymmetric information and problems in accessing markets of capital.

Second, firms are less willing to finance  since the risk of R&D investments is higher and the importance of  R&D is less clear for companies. Moreover, the asymmetric information and the fear of leaks of their idea to rival agents keep them from sharing their knowledge with financial intermediaries. from another angle, the intermediaries may be eager to reward the tangible investments to keep the chance of being colleteral  rather than intagible ones ????whose profits come in futur???????.






effective, ineffective
ubsidy,grant, incentive
reward, subsidize,privately financed
undertaken, made, triggered, activate
recipient firms
R&D spending, 

Third, even the professional public committee might not be able to differentiate between marginal and inframarginal projects due to lack of all necessary information , which decreases the effectiveness of program. Furthermore, government institutions might be applead to funding inframarginal since the probability of their success is higher and they can convince publict opiniion about the effectiveness of the policy.
Besides the abovementioned directs impacts, there are several positive indirect effects producing the crowding in to increase the potential outcome. the assignment of grant may demonstrate the projects profitabilita reducing the as info and cost of capital . In addition, bz means of the public funds fidms could upgrade their eqipments and hire wellskilled researched which benefit current and subsequent projects leading to an increase in future  profit streams. 

on the other side of negetive indirect effects  generate the crowding out.In presence of price inelasiticiz of r and d input supplz. the dmand shift for input would onlz raise the cost of projrcts. 
Considering abovementioned arguments, the effectiveness of grant would be an emprical question to answer.


##          A. Empirical Evidence

The  review  of  the  eprical literature on the effects of R&D subsidies shows the major challenge  in evaluation of policy is that that the firms in treatment and control groups are not randomly chosen. That is, there are crucial unobservable characteristics in correlation with outcome variable which differentiate the fundend companies from non-funded ones, therefore, the identifying feature for subsidy assignment is enfogenout. the paper argues that recent researches addressed the enfogenity problem through the matching or instrumenta variable methods and the results are usually sensentive to the selection of methodology.

The endogeneity problem has been addressed in recent analyses mainly through matching methods or instrumental variable estimates. 
However, irrespective of the strategy adopted, the conclusions of earlier studies are mixed. Surveying firm-level analyses conducted in the previous three decades, David, Hall, and Toole (2000) observe that almost one-half (9 out of 19) of the policies were not found to trigger additional investment while for the other half the opposite
was true. More recent evidence is similarly inconclusive. In the case of the Small Business Innovation Research program in the United States, two studies reach opposite conclusions. Matching subsidized and unsubsidized firms by industry and size, Lerner (1999) finds that the policy increased the sales and employment of subsidized firms. By contrast, Wallsten (2000), using the amount of public funds available for each type of R&D investment in each year as an instrument for the subsidy, shows that grants did not lead to an increase in employment and that the public
subsidy crowded out firm-financed R&D dollar for dollar. The evidence available for other countries is also mixed. For Israel, Lach (2002) finds that grants createdadditional R&D investment for small firms but, since the greatest share of the subsidies was given to large firms that did not make additional investment, the overall impact was null. He compared the performance of subsidized and nonsubsidized firms using difference-in-differences (DID) estimates and controlling for severalobservables. Almus and Czarnitzki (2003) use matching strategies to study R&D subsidies in Eastern Germany, finding an overall positive and significant effect oninvestment. González, Jaumandreu, and Pazó (2005) examine the effects of R&Dpolicies in Spain, estimating simultaneously the probability of obtaining a subsidy,
assuming a set of firms’ observables as predetermined (e.g., size, age, industry, location, capital growth), and the impact of the grant on investment. They find a positive,albeit very small, effect on private investment that turns out to be significantly largerfor small firms. Combining the matching method with DID estimations, Görg and Strobl (2007) find that in Ireland only small grants had additional effects on private R&D investment, while large grants crowded out private investment. Hussinger(2008) uses two-step selection models to show that in Germany public subsidies
were effective in promoting firms’ R&D investment. Finally, Jacob and Lefgren (2011) use a similar method to ours to estimate the impact of public grants on US researchers’ output measured by the number of published articles and citations, and find a limited impact of public support. Meanwhile Takalo, Tanayama, and Toivanen
(2013), using a structural model estimated on firm-level data from Finland, find positive general equilibrium effects of the subsidies on expected welfare—i.e., the expected benefits of the program net of its costs—although the expected effects of the incentives are highly heterogeneous.




The geographic area covered by the policy isdescribed in Figure A1 in the Appendix.

## II. The Program


In 2003, the inauguration of the “Regional Program for Industrial Research, Innovation and Technological Transfer,” by the govenrment of Emilia-Romagna leads to implementation of the public financing R&D program  according to the Article 4 of Regional Law 7/2002. The goal of program is to get behind the enterprises' research in the region and help them with the precompetitive development activities _the activity necessary to convert the output of research
into a plan, project, or design for the realization of new products or processes or
the improvement of existing ones(Bronzinin and lachini)_. The plan requie the regional
government to funds eligible firms for their R&D expenditures. The subsidy is planned to cover the expenses of research projects up to 50 percent and precompetitive development projects up to 25 percent; there might be an extention of 10 percent for precompetitive develoment activities for small- or medium-sized enterprises. the maximum fund woudl be €250,000. the program period  is from 1 to 2 years but extendable. the subsidy transfer could be done in two ways: one payment at the end of project or in two installments the first at the halfway of project and the second at the time of completion.
the projects which are eligible to be subsidized are as follow: (i) costs
for machinery and equipment; (ii) software; (iii) purchase and registration of
patents and licenses; (iv) employment of researchers; (v) the use of laboratories;
(vi) contracts with research centers; (vii) consulting; (viii) feasibility studies; and
(ix) external costs for the realization of prototypes.(Bronzinin and lachini) 


One critical issue is that whether the effect of regional program could be mixed with other kinds of public subsidies.However, this problem would be addressed bz the fact that each project could onlz receive one tzpe of public grant. In addition, the probability of subsiidy assignment is independent from the amount of demanded fund.  The regional government designates a committe of independent professionals to assign a score for each of aspects mentioned below: 
(i) technological and scientific (max. 45 points); (ii) financial and economic (max.
20 points); (iii) managerial (max. 20 points); and (iv) regional impact (max.
15 points).7 Only projects deemed sufficient in each category and which obtain a
total score of at least 75 points receive the grants (the maximum score is 100). For
the evaluation process, both the committee and the independent evaluators must
comply with the general principles for the evaluation of research specified by the
Ministry of Education, University and Research of the Italian Government, and the
general principles of the European Commission.
there were two rounds of applications, deadlines and evaluation process.the size of grants for industrial firms which are used for the estimation of their paper was €182,000 on average.


##                               III. Empirical Strategy and Data

###                                 A. Empirical Strategy

 As described before, the difficulty of the program's evaluation is the endogenity of characteristic which identifies the recipient firms since the difference of treated and untreated agents are related to unobserved features correlated with the response variable. Bronzini and lachini utilized the mechanism of the funds’ assignment to address the endogenity problem. That is, the committee graded each project and only thosereceiving a score greater than or equal to a threshold of 75 points out of 100  won subsidies.

For performance comparison of subsidized and nonsubsidized enterprises with scores nearby the threshold, the paper applies a sharp regression discontinuity (RD) design. Their reason to apply sharp RDD is that the the treatment status ( $T_{i}$ ) is deterministic and discontinuous function of the paper's running variable which is score with discontinuity at the cutoff score(75 point).

 
\begin{equation}
 T_{i} = \begin{cases}
       1 & \text{if score $\geq$ 75}  \\
       0  & \text{if score < 75}  
     \end{cases}
\end{equation}

The outcome variable is set to be a function of the score. In this way, the program's average treatment effect (ATE) is evaluated through the value of the discontinuity estimated at the cutoff point.

Reasoning that the utilization of RDD is growing in quasi-experimental studies in economics after ( Angrist and Lavy (1999);
Black (1999); and van der Klaauw (2002)), Bronzini and lachini arque that the RD method performs better to contril the treatment endogeneity rather than other nonexperimental strategies. Their reason is that under satisfied conditions, is is likely to demonstrate that there is a randomized experiment around the cutoff point. knowing that  the score (running variable) is not fully manipulatable around th ethreshold. The randomness aroung the threshold could be detected by looking at the densitz of the score (McCrary 2008). In addition, they test the randomization assuption by checking whether  the subsidized and non subsidiezed firms around the thereshold are similar enough or in another way of saying by verifying if observable differences of two groups become negligble. The similarity of the two groups is a consequence of randomization and
not vice versa (Lee 2008).

In absence of a direct way of testing the validity of the continiuty hypothesis claiming that covariates,  they refer to Lee (2008) arguing formally that  the continuity assumption is satisfied if the treatment relies on whether a running variable surpasses a certain threshold and that running variable is not under the control of the agent. under these conditins, the treatment's variation is randomized around the cutoff meaning that the agents are randomly drawn just above and just below the threshold and the effect of program could be identified by the discontinuity of the outcome variable at the cutoff point (Hahn, Todd,and van der Klaauw 2001). However, they present an indirec way for the robustness check of continiuty assumption in Section V by assessing whether observable and unobservable variables  correlated with the outcome variable or outcome variable in the absence of program are continuious across the RD threshold. In their case they test that the firms close to cutoff point have the same identical potential outcome in a qeivalent subsidy experience.



Regarding the susebtibility of RDD model to the choice of the functional form or the threshold interva in the local regressions, they check the robustness of their model by using different functional forms and econometric models. By refering to (see, amongst others: Imbens and Lemieux 2008; Lee and Lemieux 2010)they utilize the several test for the threshold discontinuity.


They use both parametric and nonparametric models.
In try of parametric models, first, They estimate different polynomial models form zero to third ordered ones over the full sample:

\begin{equation}
(1) Y_{i} = \alpha + \beta T_{i} + (1 − T_{i}) \sum_{i = 1}^{3} \gamma_{i} (S_{i})^p + T_{i}\sum_{i = 1}^{3}\gamma_{i}' (S_{i})^p + \epsilon_{i}
\end{equation}

where the outcome variable is denoted by $Y_{i}$; $T_{i} = 1$ if company is funded because of being scored above or equal to 75 and $T_{i} = 0$ otherwise; S_{i} = Score_{i} − 75; the parameters of grade's function are denoted by γ_{p} and γ_{i}′ on both sides of the threshold with possible different value to address the function heterogeneity across the cuttoff point; 
; and εi is the error term. They run the polynomial of order zero to check the mean difference between treated and untreated enterprises. 
Second, two different sample windows are considered to estimate the equation (1) by local regressions aaround threshold. The wide window contains half of the baseline sample (scores in the 52 to 80 range); the narrow window consists of 35 percent of the full sample (companies with scores between 66 and 78). The reason of chosen ranges is to almaost balance the number of companies above and below of threshold. the number of observations in the neigbourhood of cutoff point is relativelz low  ( 171 firms in wide window  and 115 in narrow widnow )which leads to imprecised estimation of higher-order polynomial models(see Lee and Lemieux 2010), to deal wit this issue thez estmate up to 2nd order of  the polznomila for the
local regressions around the cutoff.
Third, they use the nonparametric techniques named the Epanechnikov kernel regressions  with two bandwidths of 30 and 15  to estimate the discontinuity (the results are presented and discussed in Section V).


the correct specification of equation 1 leads to estmation of parameter Beta trhrough OLS model, the beta is measuring the amount of the functionÄs discontinuitz  at the threshold which is correspond to the unbiased estimate of programäs causal effect.however, the random error might be correlated within the group not unlike the cases discussed by Moulton 1990)dbecause of discrete  forcing variable 
 meaning that the grade could be 
 only integer value.
 
 since the groups close the dutoff receive the same score. the standard errors are underestimated(downward-biased) and might significanebut spurius siginifance occure. to deal with that thez clustere the the heteroskesadticitz robust standard errors bz the normalize score S
In our study, the groups are represented by firms that received the same score. moreover , in kernel regressions thez cluster and bootstrap the standard errors.





### B. outcome Variables and Data

   the usual suspect for the outcome variable is the amount of firm's R&D investment. However, due to the unavailability of data of R&D since expenditure, they construct their analysis based on the data of almost all Italian corporation's balance-sheets gathered by Cerved group.
    
   They take the reimbursable outlays as outcome variables. Their aim is to observe whether a significant raise in at least one of mentioned outcome variables for subsidized firms happen, comparing to nonsubsidized companies. The noticable increase in those item could  demonstrate that the program made some outlays possible that would not have been made withut public fund.


The main reimbursable expenditures are tangible or intangible assets, such as costs for machinery and quipment, software, patents, and licenses. So they take the net tangible investment as their frist and foremost outcome variable. They argue that the net intangible investment would be another outcome variable because in Italy 40 percen of innovation projects are included in tangible assets(Istituto Nazionale di Statistica 2010). The net investment is annual diffrence in tangible or intangible assets net of amortization, which drived from the balance sheet. 


The paper builds the evaluation only on total amount of intangible and tangible assets for three reasons. The first reason is inconsistency of different firms' financial reports. Only some large firms provide the Italian Civil Code with the detailed financial statment and  other firms usually smaller ones merely report the total amount of financial items. Furthermore, the information on total items are more precise than detailed ones. Also, based on a sample of five firms, it was shown that eligible expenditures for subsidies(R&D, patents, software and other intellectual property rights, licences, trademark, and ongoing intangible assets) consist of 66 percent of the total intangible assets and goodwill which is not covered by subsidy occupies 22 percent of intangible assests; therefore, despite being aware of a second-
order bias by goodwill, the reason that the total intangible  and tangible investments are reasonable outcome variables
 

Moreove, knowing that employment of researches are included in other reimbursable outlays, they consider other three outcome variables: labor costs, level of employment, and wages. From theory point of view , the program leads to substitution of researches (high skilled employees) for low-skilled workers and subsequently an increase in labor costs. Labor costs divided by the number of employees gives the wages. if wages increases the demand for higher_skilled employees goes up and program benefited the employees as shown by Goolsbee (1998).

IN addtion,  the firm buys some servises or pays for laboratories, contracts with
research centers, consulting, feasibility studies, and external costs for the realization
of prototypes for R&D projects. for this reason , service costs are considered as another outcome variable. With this strategy thez study the effect of program on different kinds of r and d spendings.

Wrap it up, they evaluate the program's effect on the following outcome variables: investment (total, tangible, and intangible), labor costs, employment, wages and service costs. 

The expected period of project's realization is three years( the year of assignment plus two next years) to detect all potential changes. Also, to avoid the potential endogeneity, all outcome variables are scaled pre-program sales(the first year before te program). The employment and wages are used as log form. Finally, to prevent the outliers to drive the results because of volatility of investment during the time and unevenness of variables across the firms , they trimmeds the sample trimmed
the sample according to the fifth and ninety-fifth percentile of the distribution of Total investmenti/Pre-programsalesi 
(Bronyini lachini, 2014). 1,2046 compnaies submitted their proposals and 557 were chosen to be treatd  and 689 to be untreated. but amon 689 untreated firms 411 of them were excluded in the second year because their projects were inefficient. the paper claims that their exclusion does not affect on the study since even if they recieved the score, they would have been far from the cutoff point whose discontinuity matters to be tested. therefore, their ommision does not make any bias. they also omit the statups and very small enterprises which are underrepresented. after cleaning the sample, the full sample with major share of grants becomes 357 industrial firms (254 treated and 103 untreated) and 111 service firms
(of which 61 were treated).
  Because of considerable heterpgeneity of industrial and service firms which might generate nhuge noise, they concentreated on industrial firms ( the vast majority was manufacturing and construction)which compares homogenous enterprises

the number of subsidized firms are almost doubled that of nonsubsideyed because nonscored firms in the second round were excluded. However, the within sector 's proportion is balanced.
 

(((((From the balance sheets we take as outcome variables those items that are associated with the expenditures reimbursable by the program listed in Section II.))))

In Figure 1 the density function of the sample by score is shown. The density is
higher on the right-hand side of the threshold because of the cited exclusion of nonscored untreated firms in the second round and increases substantially around the
cutoff point. We observe, however, that just below the cutoff (score = 74) it is lower
than at slightly more distant values.
We do not interpret this drop as the signal that firms just below the threshold
were able to manipulate their score. Rather, we believe that the committee of experts
avoided assigning a score just below the threshold for understandable reasons. A
similar outcome could have been perceived as particularly annoying by unsuccessful applicant firms and would potentially have left more room for appeals against the
decision. If anything, this evidence shows that the committee enjoys a certain degree
of discretion in assigning the score, a characteristic of the assessment that does not
invalidate our design.
IV. Results
A. Baseline Results
We first present the estimations of the coefficient β of model (1) using total, tangible, and intangible investment scaled by pre-program sales as outcome variables.
Since we do not observe privately financed investment separately from that financed
by public incentive, we shall now briefly discuss how to interpret the results. A coefficient β equal to zero would signal complete crowding-out of private investment
by public grants: firms reduced private expenditure by the amount of the subsidies
received and the investment turned out to be unaffected by the program. On the other
hand, a positive coefficient would show that overall treated firms invested more than
untreated firms, plausibly thanks to the program, and that total crowding-out did not

occur. However, it is still possible that firms partially substituted public for privately
financed R&D outlays. In order to evaluate if partial crowding-out, or on the contrary,
even crowding-in, occurred—that is if public subsidies triggered privately financed
investment—we have to compare the change in total investment with the grants.
Before showing the econometric results let us present the scatterplot of the (averaged by score) outcome variables against the score (Figure 2). As expected, the
figure shows rather dispersed points, given that investment is usually greatly uneven
across firms. The interpolation lines appear almost flat, showing a weak dependence
of the overall outcome on the score. As a matter of fact, no remarkable jumps of the
outcome variables at the threshold emerge from the figures; however, if anything,
the impact seems somewhat positive.
This perception is confirmed by the econometric estimates of the coefficient β
for total, tangible, and intangible investment shown in Table 3. Akaike Information
Criterion (AIC) suggests a preference for more parsimonious models, namely simple
mean differences, rather than a higher order of polynomials in all cases but one. The
sign of the coefficient is almost always positive. Using the full sample as a benchmark, the jump turns out to be equal to about one-third of the mean of the outcome
variable of the untreated firms. Due to the sample variance, however, the discontinuity
is almost never statistically significant (the coefficient is weakly significant in just 4
out of 30 models). Local estimates generate similar results to those of the full sample.
It is possible that we were unable to detect any effect because, for example, firms
had used the grants for hiring researchers or for consulting contracts. To check for
this eventuality we test for discontinuity of labor and service costs, using these as
additional outcome variables. Furthermore, we change the scale variable for investment using capital and total assets (calculated in the pre-program year) to check the
sensitiveness of our previous findings on investment. Both are taken from firms’
balance sheets. Capital is defined as the sum of tangible and intangible assets (fixed
assets). Total assets are the sum of fixed, current, and other assets. The results of
these exercises are reported in Table 4.
Labor costs almost always have a negative sign, but only rarely is the coefficient
statistically significant. With regard to service costs, the discontinuity is never significant and the sign is not stable across the model’s specifications. The previous
results do not even appear affected by the variable used to scale investment, although
in some models the coefficient now turns out to be statistically significant. Finally,
we estimated the effect of the incentives on the (log of) employment and wages for
a subsample of firms that reported information on employment (263 out of 357).19
The former aims to ascertain the effect of the policy on firms’ employment; the latter to verify whether the benefits of the program went mainly to employees through
higher wages, as shown for the United States by Goolsbee (1998). Table B2 in
the online Appendix displays the results (in the local regressions we use only the
wide-window because of the narrower sample size). Overall it seems that neither
the level of employment nor that of wages changed thanks to the program: the coefficients are almost never statistically significant.

occur. However, it is still possible that firms partially substituted public for privately
financed R&D outlays. In order to evaluate if partial crowding-out, or on the contrary,
even crowding-in, occurred—that is if public subsidies triggered privately financed
investment—we have to compare the change in total investment with the grants.
Before showing the econometric results let us present the scatterplot of the (averaged by score) outcome variables against the score (Figure 2). As expected, the
figure shows rather dispersed points, given that investment is usually greatly uneven
across firms. The interpolation lines appear almost flat, showing a weak dependence
of the overall outcome on the score. As a matter of fact, no remarkable jumps of the
outcome variables at the threshold emerge from the figures; however, if anything,
the impact seems somewhat positive.
This perception is confirmed by the econometric estimates of the coefficient β
for total, tangible, and intangible investment shown in Table 3. Akaike Information
Criterion (AIC) suggests a preference for more parsimonious models, namely simple
mean differences, rather than a higher order of polynomials in all cases but one. The
sign of the coefficient is almost always positive. Using the full sample as a benchmark, the jump turns out to be equal to about one-third of the mean of the outcome
variable of the untreated firms. Due to the sample variance, however, the discontinuity
is almost never statistically significant (the coefficient is weakly significant in just 4
out of 30 models). Local estimates generate similar results to those of the full sample.
It is possible that we were unable to detect any effect because, for example, firms
had used the grants for hiring researchers or for consulting contracts. To check for
this eventuality we test for discontinuity of labor and service costs, using these as
additional outcome variables. Furthermore, we change the scale variable for investment using capital and total assets (calculated in the pre-program year) to check the
sensitiveness of our previous findings on investment. Both are taken from firms’
balance sheets. Capital is defined as the sum of tangible and intangible assets (fixed
assets). Total assets are the sum of fixed, current, and other assets. The results of
these exercises are reported in Table 4.
Labor costs almost always have a negative sign, but only rarely is the coefficient
statistically significant. With regard to service costs, the discontinuity is never significant and the sign is not stable across the model’s specifications. The previous
results do not even appear affected by the variable used to scale investment, although
in some models the coefficient now turns out to be statistically significant. Finally,
we estimated the effect of the incentives on the (log of) employment and wages for
a subsample of firms that reported information on employment (263 out of 357).19
The former aims to ascertain the effect of the policy on firms’ employment; the latter to verify whether the benefits of the program went mainly to employees through
higher wages, as shown for the United States by Goolsbee (1998). Table B2 in
the online Appendix displays the results (in the local regressions we use only the
wide-window because of the narrower sample size). Overall it seems that neither
the level of employment nor that of wages changed thanks to the program: the coefficients are almost never statistically significant.

In [96]:
SC_pol0 = ols("SCSALES ~ T ", data = df_narr_wind).fit(cov_type='cluster', cov_kwds={'groups': df_narr_wind["score"]}).summary()

In [97]:
SC_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                SCSALES   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                 -0.008
Method:                 Least Squares   F-statistic:                   0.05725
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.815
Time:                        21:34:31   Log-Likelihood:                -97.448
No. Observations:                 115   AIC:                             198.9
Df Residuals:                     113   BIC:                             204.4
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0086      0.057     17.580      0.000       0.896       1.121
T             -0.0254      0.106     -0.239      0.811      -0.233       0.182
==============================================================================
Omnibus:                      105.205   Durbin-Watson:                   2.056
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1164.822
Skew:                           3.104   Prob(JB):                    1.15e-253
Kurtosis:                      17.302   Cond. No.                         2.61
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [98]:
ols_pol0 = ols("INVSALES ~ T", data= df).fit(cov_type='cluster', cov_kwds={'groups': df["score"]}).summary()

In [99]:
ols_pol0 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                    0.8927
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.349
Time:                        21:34:31   Log-Likelihood:                 301.50
No. Observations:                 357   AIC:                            -599.0
Df Residuals:                     355   BIC:                            -591.2
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0333      0.011      3.111      0.002       0.012       0.054
T              0.0122      0.013      0.945      0.345      -0.013       0.038
==============================================================================
Omnibus:                       63.908   Durbin-Watson:                   1.825
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              103.114
Skew:                           1.064   Prob(JB):                     4.07e-23
Kurtosis:                       4.550   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [100]:
ols_pol0 = ols("INVSALES ~ T ", data= df).fit(cov_type='cluster', cov_kwds={'groups': df["score"]}).summary()

In [101]:
ww_pol222 = ols("INVINTSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data = df_wide_wind).fit(cov_type='cluster', cov_kwds={'groups': df_wide_wind["score"]}).summary()


In [102]:
ww_pol222

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            INVINTSALES   R-squared:                       0.045
Model:                            OLS   Adj. R-squared:                  0.016
Method:                 Least Squares   F-statistic:                     3.776
Date:                Tue, 23 Jun 2020   Prob (F-statistic):             0.0115
Time:                        21:34:31   Log-Likelihood:                 237.36
No. Observations:                 171   AIC:                            -462.7
Df Residuals:                     165   BIC:                            -443.9
Df Model:                           5                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0304      0.017     -1.808      0.071      -0.063       0.003
T              0.0735      0.024      3.035      0.002       0.026       0.121
streat        -0.0250      0.012     -2.046      0.041      -0.049      -0.001
snotreat      -0.0101      0.005     -1.973      0.049      -0.020   -6.49e-05
s2treat        0.0044      0.002      1.901      0.057      -0.000       0.009
s2notreat     -0.0004      0.000     -1.837      0.066      -0.001    2.78e-05
==============================================================================
Omnibus:                       81.323   Durbin-Watson:                   1.624
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              442.428
Skew:                           1.695   Prob(JB):                     8.47e-97
Kurtosis:                      10.114   Cond. No.                         692.
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

Firm
0      1.0
1      0.0
2      1.0
3      1.0
4      0.0
      ... 
352    1.0
353    1.0
354    1.0
355    1.0
356    1.0
Name: LF, Length: 357, dtype: float64

In [117]:
ISF_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                    0.8927
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.349
Time:                        22:09:16   Log-Likelihood:                 301.50
No. Observations:                 357   AIC:                            -599.0
Df Residuals:                     355   BIC:                            -591.2
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0333      0.011      3.111      0.002       0.012       0.054
T              0.0122      0.013      0.945      0.345      -0.013       0.038
==============================================================================
Omnibus:                       63.908   Durbin-Watson:                   1.825
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              103.114
Skew:                           1.064   Prob(JB):                     4.07e-23
Kurtosis:                       4.550   Cond. No.                         3.49
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [119]:

df_large = df.loc[df["SIZE"]=="large", :]
#df_large

df_small = df.loc[ df["SIZE"] == "small" , : ]
#df_small["s"]

In [120]:
ISS_pol0 = ols("INVSALES ~ T", data = df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ISS_pol1 = ols("INVSALES ~ T + streat + snotreat ", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ISS_pol2 = ols("INVSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ISS_pol3 = ols("INVSALES ~ T +  +streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()


ISL_pol0 = ols("INVSALES ~ T", data = df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ISL_pol1 = ols("INVSALES ~ T + streat + snotreat ", data =  df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ISL_pol2 = ols("INVSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ISL_pol3 = ols("INVSALES ~ T +  +streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data =  df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()

In [121]:
#ISS_pol0
#ISS_pol1
#ISS_pol2
#ISS_pol3

In [130]:
ISS_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.032
Method:                 Least Squares   F-statistic:                     6.028
Date:                Tue, 23 Jun 2020   Prob (F-statistic):             0.0179
Time:                        22:13:31   Log-Likelihood:                 142.59
No. Observations:                 178   AIC:                            -281.2
Df Residuals:                     176   BIC:                            -274.8
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0229      0.014      1.612      0.107      -0.005       0.051
T              0.0452      0.018      2.455      0.014       0.009       0.081
==============================================================================
Omnibus:                       32.420   Durbin-Watson:                   1.845
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               45.239
Skew:                           1.058   Prob(JB):                     1.50e-10
Kurtosis:                       4.274   Cond. No.                         3.24
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [131]:
ISL_pol0

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.004
Method:                 Least Squares   F-statistic:                     1.166
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.286
Time:                        22:13:31   Log-Likelihood:                 166.46
No. Observations:                 179   AIC:                            -328.9
Df Residuals:                     177   BIC:                            -322.5
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0473      0.018      2.561      0.010       0.011       0.084
T             -0.0217      0.020     -1.080      0.280      -0.061       0.018
==============================================================================
Omnibus:                       31.899   Durbin-Watson:                   2.344
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               49.023
Skew:                           0.962   Prob(JB):                     2.26e-11
Kurtosis:                       4.695   Cond. No.                         3.81
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
"""

In [ ]:
ISL_pol2

In [126]:
ISL_pol3

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               INVSALES   R-squared:                       0.034
Model:                            OLS   Adj. R-squared:                 -0.005
Method:                 Least Squares   F-statistic:                     1.780
Date:                Tue, 23 Jun 2020   Prob (F-statistic):              0.115
Time:                        22:13:31   Log-Likelihood:                 168.74
No. Observations:                 179   AIC:                            -321.5
Df Residuals:                     171   BIC:                            -296.0
Df Model:                           7                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0939      0.058      1.630      0.103      -0.019       0.207
T             -0.0302      0.064     -0.476      0.634      -0.155       0.094
streat        -0.0166      0.011     -1.521      0.128      -0.038       0.005
snotreat       0.0140      0.016      0.864      0.387      -0.018       0.046
s2treat        0.0018      0.001      1.449      0.147      -0.001       0.004
s2notreat      0.0009      0.001      1.000      0.317      -0.001       0.003
s3treat    -5.882e-05   4.03e-05     -1.460      0.144      -0.000    2.02e-05
s3notreat   1.318e-05   1.18e-05      1.118      0.264   -9.93e-06    3.63e-05
==============================================================================
Omnibus:                       30.795   Durbin-Watson:                   2.341
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               48.637
Skew:                           0.915   Prob(JB):                     2.75e-11
Kurtosis:                       4.782   Cond. No.                     1.14e+05
==============================================================================

Warnings:
[1] Standard Errors are robust to cluster correlation (cluster)
[2] The condition number is large, 1.14e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [ ]:
# tangible investment/presales program by firm's size
ITS_pol0 = ols("INVTSALES ~ T", data = df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ITS_pol1 = ols("INVTSALES ~ T + streat + snotreat ", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ITS_pol2 = ols("INVTSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ITS_pol3 = ols("INVTSALES ~ T +  +streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()


ITL_pol0 = ols("INVTSALES ~ T", data = df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ITL_pol1 = ols("INVTSALES ~ T + streat + snotreat ", data =  df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ITL_pol2 = ols("INVTSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ITL_pol3 = ols("INVTSALES ~ T +  +streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data =  df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()

In [ ]:
ITS_pol0

In [ ]:
ITS_pol1

In [ ]:
ITS_pol2

In [ ]:
ITS_pol3

In [ ]:
ITL_pol0

In [ ]:
ITL_pol1

In [ ]:
ITL_pol2

In [ ]:
ITL_pol3

In [ ]:
df_narr_small = df.loc[(66 <=df.score) & (df.score <= 78) & (df.SIZE == "small") , :]
df_narr_large = df.loc[(66 <=df["score"]) & (df["score"] <= 78) & (df["SIZE"] == "large") , :]

df_wide_small = df.loc[(52 <=df.score) & (df.score <= 80) & (df.SIZE == "small") , :]
df_wide_large = df.loc[(52 <=df["score"]) & (df["score"] <= 80) & (df["SIZE"] == "large") , :]


In [ ]:
# total investment/pre-sales program by firm's size
ITSW_pol0 = ols("INVSALES ~ T", data = df_wide_small).fit(cov_type='cluster', cov_kwds={'groups': df_wide_small["score"]}).summary()
ITSW_pol1 = ols("INVSALES ~ T + streat + snotreat ", data = df_wide_small).fit(cov_type='cluster', cov_kwds={'groups': df_wide_small["score"]}).summary()
ITSW_pol2 = ols("INVSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data = df_wide_small).fit(cov_type='cluster', cov_kwds={'groups': df_wide_small["score"]}).summary()




In [ ]:
ITSW_pol0 

In [ ]:
ITSW_pol1

In [ ]:
ITSW_pol2

In [ ]:
ITLW_pol0 = ols("INVSALES ~ T", data = df_wide_large).fit(cov_type='cluster', cov_kwds={'groups': df_wide_large["score"]}).summary()
ITLW_pol1 = ols("INVSALES ~ T + streat + snotreat ", data =  df_wide_large).fit(cov_type='cluster', cov_kwds={'groups': df_wide_large["score"]}).summary()
ITLW_pol2 = ols("INVSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_wide_large).fit(cov_type='cluster', cov_kwds={'groups': df_wide_large["score"]}).summary()


In [ ]:
ITLW_pol0

In [ ]:
ITLW_pol1

In [ ]:
ITLW_pol2

In [ ]:
# tangible investment/presales program by firm's size
ITS_pol0 = ols("INVTSALES ~ T", data = df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()
ITS_pol1 = ols("INVTSALES ~ T + streat + snotreat ", data =  df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()
ITS_pol2 = ols("INVTSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()


ITL_pol0 = ols("INVTSALES ~ T", data = df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()
ITL_pol1 = ols("INVTSALES ~ T + streat + snotreat ", data =  df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()
ITL_pol2 = ols("INVTSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()


In [ ]:
ITS_pol0

In [ ]:
ITS_pol1

In [ ]:
ITS_pol2

In [ ]:
#Intangible investment/pre-program sales in narrow window sample
IINTS_pol0 = ols("INVINTSALES ~ T", data = df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()
IINTS_pol1 = ols("INVINTSALES ~ T + streat + snotreat ", data =  df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()
IINTS_pol2 = ols("INVINTSALES ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()


IINTL_pol0 = ols("INVINTSALES ~ T", data = df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()
IINTL_pol1 = ols("INVINTSALES ~ T + streat + snotreat ", data =  df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()
IINTL_pol2 = ols("INVINTSALES ~ T +streat + snotreat + s2treat + s2notreat", data =  df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()


In [ ]:
ITS_pol0

In [ ]:
IINTS_pol1

In [ ]:
IINTS_pol2

In [ ]:
IINTL_pol0

In [ ]:
IINTL_pol1

In [ ]:
IINTL_pol2

In [ ]:
# investment full sample
IS_pol0 = ols("INV ~ T", data = df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
IS_pol1 = ols("INV ~ T + streat + snotreat ", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
IS_pol2 = ols("INV ~ T +  +streat + snotreat + s2treat + s2notreat", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()


# investment full sample
ISF_pol0 = ols("INV ~ T", data = df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ISF_pol1 = ols("INV ~ T + streat + snotreat ", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ISF_pol2 = ols("INV ~ T +streat + snotreat + s2treat + s2notreat", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()
ISF_pol2 = ols("INV ~ T +streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()

# investment full sample
ILF_pol0 = ols("INV ~ T", data = df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ILF_pol1 = ols("INV ~ T + streat + snotreat ", data = df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ILF_pol2 = ols("INV ~ T +streat + snotreat + s2treat + s2notreat", data = df_large).fit(cov_type='cluster', cov_kwds={'groups': df_large["score"]}).summary()
ISF_pol2 = ols("INV ~ T +streat + snotreat + s2treat + s2notreat + s3treat + s3notreat", data =  df_small).fit(cov_type='cluster', cov_kwds={'groups': df_small["score"]}).summary()

#investment in narrow window sample
IS_pol0 = ols("INV ~ T", data = df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()
IS_pol1 = ols("INV ~ T + streat + snotreat ", data =  df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()
IS_pol2 = ols("INV ~ T +streat + snotreat + s2treat + s2notreat", data =  df_narr_small).fit(cov_type='cluster', cov_kwds={'groups': df_narr_small["score"]}).summary()

#investment in wide window sample
IL_pol0 = ols("INV ~ T", data = df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()
IL_pol1 = ols("INV ~ T + streat + snotreat ", data =  df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()
IL_pol2 = ols("INV ~ T +streat + snotreat + s2treat + s2notreat", data =  df_narr_large).fit(cov_type='cluster', cov_kwds={'groups': df_narr_large["score"]}).summary()


In [ ]:
IS_pol1

In [ ]:
df["sizem"] = np.nan
df.loc[df["SIZE"] == "small", "sizem"] = 1
df.loc[df["SIZE"] == "large", "sizem"] = 0

df["SF"] = df["sizem"]
df["LF"] = 1 - df["sizem"]
#df_small = df.loc[df["sizem"] == 1 , : ]
#df_small["SF"] = df


In [ ]:
df["largem"]  = np.nan
df["slarge"]  = np.nan
df["s2large"] = np.nan
df["s3large"] = np.nan

df["largem"]  = df["LF"]
df["slarge"]  = df["s"] * df["largem"]
df["s2large"] = df["s"].pow(2) * df["largem"]
df["s3large"] = df["s"].pow(3) * df["largem"]

In [ ]:
df["smallem"]  = np.nan
df["ssmallem"]  = np.nan
df["s2smallem"] = np.nan
df["s3smallem"] = np.nan

df["smallem"]  = df["SF"]
df["ssmallem"]  = df["s"] * df["smallem"]
df["s2smallem"] = df["s"].pow(2) * df["smallem"]
df["s3smallem"] = df["s"].pow(3) * df["smallem"]

In [ ]:
# total investment/pre-sales program by firm's size
ISF_pol0 = ols("INVSALES ~ T", data = df).fit(cov_type='cluster', cov_kwds={'groups': df["score"]}).summary()
ISF_pol1 = ols("INVSALES ~ T + ssmallem + slarge", data = df).fit(cov_type='cluster', cov_kwds={'groups': df["score"]}).summary()
ISF_pol2 = ols("INVSALES ~ T +ssmallem + slarge + s2smallem + s2large", data = df).fit(cov_type='cluster', cov_kwds={'groups': df["score"]}).summary()
ISF_pol2 = ols("INVSALES ~ T +ssmallem + slarge + s2smallem + s2large + s3smallem + s3large", data = df).fit(cov_type='cluster', cov_kwds={'groups': df["score"]}).summary()